In [1]:
#libraries
import re
import pandas as pd
import numpy as np
from scipy.stats import chisquare
import pysal.model.spreg as spreg
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pysal.lib
import geopandas as gpd
import pandas as pd
from libpysal.weights import DistanceBand
from scipy.spatial import distance_matrix
import pandas as pd
from scipy.stats import pearsonr
from esda.moran import Moran
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats as stats
import spreg
from spreg import SURlagIV
from spreg import GM_Lag
import libpysal
from spreg import AKtest
from spreg import ML_Lag
from spreg import GM_Combo

In [2]:
# lets read the metadata
with pd.io.stata.StataReader(r"C:\Users\soere\OneDrive\Desktop\Matserarbeit\Python\Variablen\Classification\control.dta") as reader:
    meta_control = reader.variable_labels()
    
with pd.io.stata.StataReader(r"C:\Users\soere\OneDrive\Desktop\Matserarbeit\Python\Variablen\Independent Variables\_FINAL_c_steamenginges_1904.dta") as reader:
    meta_steam = reader.variable_labels()

In [3]:
meta_control

{'id_inequality': 'County ID',
 'rb_id': 'District ID',
 'rb_name': 'District name',
 'province_id': 'Province ID',
 's_industry': 'Share industry employment',
 's_prot': 'Share Protestants',
 's_german': 'Share German-speaking',
 'longitud': 'Longitude',
 'latitud': 'Latitude',
 'slope_km': 'Terrain slope',
 'soil_suitability': 'Soil suitability',
 'mean_temp': 'Average temperature',
 'mean_preci': 'Average precipitation',
 'dist_100km': 'Distance to the nearest river',
 'log_distance_carbon_c': 'Least distance to carbon field (in log)'}

In [4]:
# Dependent variable
dep_var_path = r"C:\Users\soere\OneDrive\Desktop\Python MA\2.2 Saving the Dependent Variables"

# Independent variable
ind_var_path = r"C:\Users\soere\OneDrive\Desktop\Python MA\1.2 Saving the Independent Variables"

# Steamengines and horsepower
power_path = r"C:\Users\soere\OneDrive\Desktop\Python MA\0. Pretransformed Data"

ifo_path = r"C:\Users\soere\OneDrive\Desktop\Python MA\0.3 ifo Data"

In [5]:
# Load Skill levels per Landkreis
skills_df = pd.read_csv(f"{dep_var_path}\Main_Dep.csv",
                        sep=",")

# Load control variables
control_df = pd.read_csv(f"{ind_var_path}\Independent Variables.csv",
                         sep=",")

# Load SteamEngine and Horsepower data
power_df = pd.read_stata(f"{power_path}\_FINAL_c_steamenginges_1904.dta")

In [6]:
# Merge the dataframes using an inner join on the "id_inequality" column
prep_df = pd.merge(control_df, skills_df, on="id_inequality", how="inner")
merged_df = pd.merge(prep_df, power_df, on="id_inequality", how="inner")

In [7]:
merged_df

,id_inequality,rb_id,rb_name_x,province_id,s_industry,s_prot,s_german,longitud,latitud,slope_km,...,unskilled,Industriearbeiter,Anteil_Industriearbeiter,per_high,per_medium,per_low,per_unskilled,steamengines_number,horsepower,rb_name_y
0,1.0,17.0,Koenigsberg,1.0,18.515817,93.379268,56.527919,21.261999,55.648117,0.039,...,14043.295736,5482.0,0.214862,0.007630,0.218746,0.223209,0.550415,236.0,4499.0,Koenigsberg
1,2.0,17.0,Koenigsberg,1.0,20.088563,96.752059,99.449711,20.249264,54.843463,0.041,...,10586.366160,4673.0,0.231245,0.009308,0.225511,0.241311,0.523870,374.0,3653.0,Koenigsberg
2,4.0,17.0,Koenigsberg,1.0,35.498406,91.659959,99.395196,20.503550,54.706534,0.041,...,22103.884188,40793.0,0.435376,0.019338,0.361893,0.382858,0.235911,485.0,8563.0,Koenigsberg
3,5.0,17.0,Koenigsberg,1.0,15.060419,97.782661,99.669951,20.646094,54.737281,0.039,...,11623.154277,3063.0,0.172408,0.005532,0.202791,0.137440,0.654236,392.0,5820.0,Koenigsberg
4,6.0,17.0,Koenigsberg,1.0,12.007056,98.668155,89.094425,21.267903,54.855057,0.038,...,13898.117010,2839.0,0.136688,0.003992,0.198231,0.128634,0.669144,166.0,1581.0,Koenigsberg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
570,698.0,1.0,Aachen,12.0,15.278715,1.613553,70.916132,6.167253,50.364714,0.143,...,9385.982854,3334.0,0.158664,0.011330,0.162573,0.379421,0.446675,89.0,1548.0,Aachen
571,699.0,31.0,Sigmaringen,14.0,19.492555,5.190840,99.167226,9.235305,48.006617,0.165,...,5951.621706,2566.0,0.217053,0.007619,0.251187,0.237757,0.503436,57.0,488.0,Sigmaringen
572,700.0,31.0,Sigmaringen,14.0,23.974037,2.518404,99.882758,9.198332,48.235600,0.186,...,2289.005948,1144.0,0.256330,0.003951,0.228216,0.254948,0.512885,16.0,81.0,Sigmaringen
573,701.0,31.0,Sigmaringen,14.0,30.325834,6.910838,99.726559,9.001089,48.317343,0.182,...,6021.065087,3900.0,0.314719,0.004489,0.267196,0.242431,0.485883,76.0,876.0,Sigmaringen


In [8]:
# Drop Erblanden
#district_Erblanden = [31]
#merged_df = merged_df[~merged_df["rb_id"].isin(district_Erblanden)]

In [9]:
merged_df

,id_inequality,rb_id,rb_name_x,province_id,s_industry,s_prot,s_german,longitud,latitud,slope_km,...,unskilled,Industriearbeiter,Anteil_Industriearbeiter,per_high,per_medium,per_low,per_unskilled,steamengines_number,horsepower,rb_name_y
0,1.0,17.0,Koenigsberg,1.0,18.515817,93.379268,56.527919,21.261999,55.648117,0.039,...,14043.295736,5482.0,0.214862,0.007630,0.218746,0.223209,0.550415,236.0,4499.0,Koenigsberg
1,2.0,17.0,Koenigsberg,1.0,20.088563,96.752059,99.449711,20.249264,54.843463,0.041,...,10586.366160,4673.0,0.231245,0.009308,0.225511,0.241311,0.523870,374.0,3653.0,Koenigsberg
2,4.0,17.0,Koenigsberg,1.0,35.498406,91.659959,99.395196,20.503550,54.706534,0.041,...,22103.884188,40793.0,0.435376,0.019338,0.361893,0.382858,0.235911,485.0,8563.0,Koenigsberg
3,5.0,17.0,Koenigsberg,1.0,15.060419,97.782661,99.669951,20.646094,54.737281,0.039,...,11623.154277,3063.0,0.172408,0.005532,0.202791,0.137440,0.654236,392.0,5820.0,Koenigsberg
4,6.0,17.0,Koenigsberg,1.0,12.007056,98.668155,89.094425,21.267903,54.855057,0.038,...,13898.117010,2839.0,0.136688,0.003992,0.198231,0.128634,0.669144,166.0,1581.0,Koenigsberg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
570,698.0,1.0,Aachen,12.0,15.278715,1.613553,70.916132,6.167253,50.364714,0.143,...,9385.982854,3334.0,0.158664,0.011330,0.162573,0.379421,0.446675,89.0,1548.0,Aachen
571,699.0,31.0,Sigmaringen,14.0,19.492555,5.190840,99.167226,9.235305,48.006617,0.165,...,5951.621706,2566.0,0.217053,0.007619,0.251187,0.237757,0.503436,57.0,488.0,Sigmaringen
572,700.0,31.0,Sigmaringen,14.0,23.974037,2.518404,99.882758,9.198332,48.235600,0.186,...,2289.005948,1144.0,0.256330,0.003951,0.228216,0.254948,0.512885,16.0,81.0,Sigmaringen
573,701.0,31.0,Sigmaringen,14.0,30.325834,6.910838,99.726559,9.001089,48.317343,0.182,...,6021.065087,3900.0,0.314719,0.004489,0.267196,0.242431,0.485883,76.0,876.0,Sigmaringen


# Merge with ifo Data

#### Preparation

In [10]:
# Load the data from the selfmade dictionary
ifo_dict_own = pd.read_excel(f"{ifo_path}\ifo Dictionary Identification.xlsx", sheet_name="Tabelle1")

In [11]:
# Merge the county_names_df dataframe with the ifo_data to add the kreiskey1901 column
ifo_df = pd.merge(merged_df, ifo_dict_own, left_on="id_inequality", right_on="Match", how="left")
ifo_df

,id_inequality,rb_id,rb_name_x,province_id,s_industry,s_prot,s_german,longitud,latitud,slope_km,...,Anteil_Industriearbeiter,per_high,per_medium,per_low,per_unskilled,steamengines_number,horsepower,rb_name_y,kreiskey1901,Match
0,1.0,17.0,Koenigsberg,1.0,18.515817,93.379268,56.527919,21.261999,55.648117,0.039,...,0.214862,0.007630,0.218746,0.223209,0.550415,236.0,4499.0,Koenigsberg,1.0,1.0
1,2.0,17.0,Koenigsberg,1.0,20.088563,96.752059,99.449711,20.249264,54.843463,0.041,...,0.231245,0.009308,0.225511,0.241311,0.523870,374.0,3653.0,Koenigsberg,2.0,2.0
2,4.0,17.0,Koenigsberg,1.0,35.498406,91.659959,99.395196,20.503550,54.706534,0.041,...,0.435376,0.019338,0.361893,0.382858,0.235911,485.0,8563.0,Koenigsberg,3.0,4.0
3,5.0,17.0,Koenigsberg,1.0,15.060419,97.782661,99.669951,20.646094,54.737281,0.039,...,0.172408,0.005532,0.202791,0.137440,0.654236,392.0,5820.0,Koenigsberg,4.0,5.0
4,6.0,17.0,Koenigsberg,1.0,12.007056,98.668155,89.094425,21.267903,54.855057,0.038,...,0.136688,0.003992,0.198231,0.128634,0.669144,166.0,1581.0,Koenigsberg,5.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
570,698.0,1.0,Aachen,12.0,15.278715,1.613553,70.916132,6.167253,50.364714,0.143,...,0.158664,0.011330,0.162573,0.379421,0.446675,89.0,1548.0,Aachen,569.0,698.0
571,699.0,31.0,Sigmaringen,14.0,19.492555,5.190840,99.167226,9.235305,48.006617,0.165,...,0.217053,0.007619,0.251187,0.237757,0.503436,57.0,488.0,Sigmaringen,570.0,699.0
572,700.0,31.0,Sigmaringen,14.0,23.974037,2.518404,99.882758,9.198332,48.235600,0.186,...,0.256330,0.003951,0.228216,0.254948,0.512885,16.0,81.0,Sigmaringen,571.0,700.0
573,701.0,31.0,Sigmaringen,14.0,30.325834,6.910838,99.726559,9.001089,48.317343,0.182,...,0.314719,0.004489,0.267196,0.242431,0.485883,76.0,876.0,Sigmaringen,572.0,701.0


In [12]:
# Load the dictionary provided by the ifo
ifo_dict = pd.read_csv(f"{ifo_path}\\ipehd_merge_county.csv")

# Give ifo_df the identifier from the ifo_dict
# using kreiskey1901 as the identifier
ifo_df = pd.merge(ifo_df, 
                  ifo_dict[["kreiskey1901", "kreiskey1800"]], 
                  on="kreiskey1901", 
                  how="left")

# Drop rows where kreiskey1800 is NaN
ifo_df.dropna(subset=["kreiskey1800"], inplace=True)

In [13]:
ifo_df.columns

Index(['id_inequality', 'rb_id', 'rb_name_x', 'province_id', 's_industry',
       's_prot', 's_german', 'longitud', 'latitud', 'slope_km',
       'soil_suitability', 'mean_temp', 'mean_preci', 'dist_100km',
       'log_distance_carbon_c', 'NAME', 'county_name', 'uni_dummy',
       'min_distance_uni', 'prov_cap_dummy', 'BER_dummy', 'density',
       'd_Hauptberuf', 'high', 'medium', 'low', 'unskilled',
       'Industriearbeiter', 'Anteil_Industriearbeiter', 'per_high',
       'per_medium', 'per_low', 'per_unskilled', 'steamengines_number',
       'horsepower', 'rb_name_y', 'kreiskey1901', 'Match', 'kreiskey1800'],
      dtype='object')

In [14]:
# Summing up 'misc1821_area' for duplicate 'kreiskey1800' values and creating a single row for each
merged_df = ifo_df.groupby("kreiskey1800")[["log_distance_carbon_c",
                                                     "s_prot",
                                                     "s_german",
                                                     "longitud",
                                                     "latitud",
                                                     "slope_km",
                                                     "soil_suitability",
                                                     "mean_temp",
                                                     "mean_preci",
                                                     "dist_100km",
                                                     "min_distance_uni"]].mean().reset_index()

skills_sum = ifo_df.groupby("kreiskey1800")[["unskilled",
                                            "low",
                                            "medium",
                                            "high",
                                            "d_Hauptberuf",
                                            "steamengines_number",
                                            "horsepower"]].sum().reset_index()

BER_dummy = ifo_df.groupby("kreiskey1800")[["BER_dummy"]].max().reset_index()

merged_df = pd.merge(merged_df, 
                  skills_sum, 
                  on="kreiskey1800", 
                  how="left")

merged_df = pd.merge(merged_df, 
                  BER_dummy, 
                  on="kreiskey1800", 
                  how="left")

# Check the first few rows of the newly created dataframe
merged_df

,kreiskey1800,log_distance_carbon_c,s_prot,s_german,longitud,latitud,slope_km,soil_suitability,mean_temp,mean_preci,dist_100km,min_distance_uni,unskilled,low,medium,high,d_Hauptberuf,steamengines_number,horsepower,BER_dummy
0,1.0,5.358620,95.806330,99.799868,20.548731,54.397921,0.050000,2.102802,7.367000,59.828000,0.373000,0.442739,34883.768083,6767.603880,12209.372807,285.255230,54146.0,679.0,4861.0,0
1,2.0,5.567290,10.590991,79.509127,20.643597,53.937462,0.063000,1.413690,7.218000,52.488000,0.379333,0.808650,43982.647569,14941.215059,17452.082295,599.055076,76975.0,403.0,3315.0,0
2,3.0,5.621669,92.770560,85.218453,19.863912,53.859214,0.059667,1.819006,7.364667,55.057667,0.095667,1.089085,37435.262831,10471.527135,14675.508755,407.701280,62990.0,878.0,5976.0,0
3,4.0,5.621901,84.495686,27.943097,20.734716,53.462761,0.067000,2.251050,7.208500,46.101000,0.447500,1.308159,30516.418375,7691.764729,10824.645456,288.171440,49321.0,390.0,3151.0,0
4,5.0,5.421339,94.876874,99.482576,21.357758,54.253761,0.052000,1.875000,7.375500,49.400500,0.191500,0.974039,19429.061925,5474.878785,7029.235783,277.823507,32211.0,414.0,3419.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,277.0,1.997243,3.062395,98.907334,6.090387,50.649088,0.109000,3.666667,8.756000,83.165000,0.667000,0.110817,4371.460958,4155.018795,2733.636857,105.883390,11366.0,220.0,4209.0,0
276,278.0,2.785375,4.405721,99.762551,6.289372,50.605630,0.138000,5.000000,7.850000,86.592000,0.555000,0.244403,4678.245341,2509.927875,1729.825572,41.001212,8959.0,54.0,799.0,0
277,279.0,3.585311,4.088966,99.687772,6.560337,50.494628,0.135000,2.500000,7.907000,81.355000,0.472000,0.531539,10651.028190,5293.003586,4607.236824,142.731400,20694.0,253.0,5686.0,0
278,280.0,3.452736,1.613553,70.916132,6.167253,50.364714,0.143000,2.785714,7.611000,93.091000,0.724000,0.400585,9385.982854,7972.780302,3416.156704,238.080140,21013.0,89.0,1548.0,0


#### Area and population data to create density

In [15]:
# Area data
ifo_area_df = pd.read_csv(f"{ifo_path}\\ipehd_1821_misc_area.csv", encoding='ISO-8859-1')
# Summing up areas for duplicates
ifo_area_df = ifo_area_df.groupby('kreiskey1800')['misc1821_area'].sum().reset_index()

# Check the first few rows of the newly created dataframe
ifo_area_df

,kreiskey1800,misc1821_area
0,1,1143405.0
1,2,1265474.0
2,3,1189582.0
3,4,1229218.0
4,5,634855.0
...,...,...
275,277,85005.0
276,278,55961.0
277,279,119590.0
278,280,276751.0


In [16]:
merged_df = pd.merge(merged_df, ifo_area_df[["kreiskey1800", "misc1821_area"]], 
                                         on="kreiskey1800", 
                                         how="left")

In [17]:
# Population data on underage in 1816 (females only until 14)
ifo_pop_1816_df = pd.read_csv(f"{ifo_path}\ipehd_1816_pop_demo.csv", encoding="ISO-8859-1")
ifo_pop_1816_df["underage_m"] = ifo_pop_1816_df["pop1816_m_un7"] + ifo_pop_1816_df["pop1816_m_7to14"] + ifo_pop_1816_df["pop1816_m_15to18"]
ifo_pop_1816_df["underage_f"] = ifo_pop_1816_df["pop1816_f_un7"] + ifo_pop_1816_df["pop1816_f_7to14"]
ifo_pop_1816_df["underage"] = ifo_pop_1816_df["underage_f"] + ifo_pop_1816_df["underage_m"]

ifo_pop_1816_df = ifo_pop_1816_df.groupby("kreiskey1800")[["underage"]].sum().reset_index()

# Check the first few rows of the newly created dataframe
ifo_pop_1816_df

,kreiskey1800,underage
0,1,25316.0
1,2,26417.0
2,3,27430.0
3,4,22179.0
4,5,22867.0
...,...,...
275,277,17441.0
276,278,6898.0
277,279,6860.0
278,280,11907.0


In [18]:
merged_df = pd.merge(merged_df, ifo_pop_1816_df, on="kreiskey1800", how="left")

In [19]:
# Population data
ifo_pop_df = pd.read_csv(f"{ifo_path}\\ipehd_1821_pop_demo.csv", encoding='ISO-8859-1')
# Summing up 'misc1821_area' for duplicate 'kreiskey1800' values and creating a single row for each
ifo_pop_df = ifo_pop_df.groupby('kreiskey1800')['pop1821_tot'].sum().reset_index()

# Check the first few rows of the newly created dataframe
ifo_pop_df


,kreiskey1800,pop1821_tot
0,1,85831.0
1,2,88389.0
2,3,90056.0
3,4,63720.0
4,5,52195.0
...,...,...
275,277,45877.0
276,278,17143.0
277,279,17439.0
278,280,29726.0


In [20]:
merged_df = pd.merge(merged_df, 
                     ifo_pop_df[["kreiskey1800", "pop1821_tot"]], 
                     on="kreiskey1800", 
                     how="left")

#merged_county_names_with_area.dropna(subset=["misc1821_area"], inplace=True)
merged_df["density"] = merged_df["pop1821_tot"] / merged_df["misc1821_area"]
merged_df

,kreiskey1800,log_distance_carbon_c,s_prot,s_german,longitud,latitud,slope_km,soil_suitability,mean_temp,mean_preci,...,medium,high,d_Hauptberuf,steamengines_number,horsepower,BER_dummy,misc1821_area,underage,pop1821_tot,density
0,1.0,5.358620,95.806330,99.799868,20.548731,54.397921,0.050000,2.102802,7.367000,59.828000,...,12209.372807,285.255230,54146.0,679.0,4861.0,0,1143405.0,25316.0,85831.0,0.075066
1,2.0,5.567290,10.590991,79.509127,20.643597,53.937462,0.063000,1.413690,7.218000,52.488000,...,17452.082295,599.055076,76975.0,403.0,3315.0,0,1265474.0,26417.0,88389.0,0.069847
2,3.0,5.621669,92.770560,85.218453,19.863912,53.859214,0.059667,1.819006,7.364667,55.057667,...,14675.508755,407.701280,62990.0,878.0,5976.0,0,1189582.0,27430.0,90056.0,0.075704
3,4.0,5.621901,84.495686,27.943097,20.734716,53.462761,0.067000,2.251050,7.208500,46.101000,...,10824.645456,288.171440,49321.0,390.0,3151.0,0,1229218.0,22179.0,63720.0,0.051838
4,5.0,5.421339,94.876874,99.482576,21.357758,54.253761,0.052000,1.875000,7.375500,49.400500,...,7029.235783,277.823507,32211.0,414.0,3419.0,0,634855.0,22867.0,52195.0,0.082216
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,277.0,1.997243,3.062395,98.907334,6.090387,50.649088,0.109000,3.666667,8.756000,83.165000,...,2733.636857,105.883390,11366.0,220.0,4209.0,0,85005.0,17441.0,45877.0,0.539698
276,278.0,2.785375,4.405721,99.762551,6.289372,50.605630,0.138000,5.000000,7.850000,86.592000,...,1729.825572,41.001212,8959.0,54.0,799.0,0,55961.0,6898.0,17143.0,0.306338
277,279.0,3.585311,4.088966,99.687772,6.560337,50.494628,0.135000,2.500000,7.907000,81.355000,...,4607.236824,142.731400,20694.0,253.0,5686.0,0,119590.0,6860.0,17439.0,0.145823
278,280.0,3.452736,1.613553,70.916132,6.167253,50.364714,0.143000,2.785714,7.611000,93.091000,...,3416.156704,238.080140,21013.0,89.0,1548.0,0,276751.0,11907.0,29726.0,0.107411


#### Education data from 1821

In [21]:
ifo_students_df = pd.read_csv(f"{ifo_path}\\ipehd_1816_edu_stud.csv", encoding="ISO-8859-1")

ifo_students_df.columns

Index(['id1816', 'kreiskey1800', 'county', 'rb', 'edu1816_pub_ele_stud_m',
       'edu1816_pub_ele_stud_f', 'edu1816_pri_ele_stud_m',
       'edu1816_pri_ele_stud_f', 'edu1816_pub_mim_stud_m',
       'edu1816_pub_mif_stud_f', 'edu1816_pri_mim_stud_m',
       'edu1816_pri_mif_stud_f', 'edu1816_pub_gel_stud_m'],
      dtype='object')

In [22]:
# Population data
ifo_students_df = pd.read_csv(f"{ifo_path}\\ipehd_1816_edu_stud.csv", encoding="ISO-8859-1")

# Middle and high school
student_types_h = ["edu1816_pub_mim_stud_m",
       "edu1816_pub_mif_stud_f", "edu1816_pri_mim_stud_m",
       "edu1816_pri_mif_stud_f", "edu1816_pub_gel_stud_m"]

ifo_students_df["high_stud"] = 0

for col in student_types_h:
    ifo_students_df["high_stud"] = ifo_students_df["high_stud"] + ifo_students_df[col]

# Primary schooling    
student_types_l = ["edu1816_pub_ele_stud_m",
       "edu1816_pub_ele_stud_f", "edu1816_pri_ele_stud_m",
       "edu1816_pri_ele_stud_f"]

ifo_students_df["low_stud"] = 0

for col in student_types_l:
    ifo_students_df["low_stud"] = ifo_students_df["low_stud"] + ifo_students_df[col]
    

ifo_students_df = ifo_students_df.groupby("kreiskey1800")[["low_stud","high_stud"]].sum().reset_index()

# Check the first few rows of the newly created dataframe
ifo_students_df

,kreiskey1800,low_stud,high_stud
0,1,7593.0,0.0
1,2,2410.0,0.0
2,3,9580.0,0.0
3,4,7249.0,0.0
4,5,6156.0,0.0
...,...,...,...
275,277,2485.0,728.0
276,278,865.0,99.0
277,279,995.0,0.0
278,280,3327.0,0.0


In [23]:
merged_df = pd.merge(merged_df, ifo_students_df, on="kreiskey1800", how="left")

In [24]:
# Calculate student share
merged_df["s_high_stud"] = merged_df["high_stud"]/merged_df["underage"]
merged_df["s_low_stud"] = merged_df["low_stud"]/merged_df["underage"]

In [25]:
merged_df

,kreiskey1800,log_distance_carbon_c,s_prot,s_german,longitud,latitud,slope_km,soil_suitability,mean_temp,mean_preci,...,horsepower,BER_dummy,misc1821_area,underage,pop1821_tot,density,low_stud,high_stud,s_high_stud,s_low_stud
0,1.0,5.358620,95.806330,99.799868,20.548731,54.397921,0.050000,2.102802,7.367000,59.828000,...,4861.0,0,1143405.0,25316.0,85831.0,0.075066,7593.0,0.0,0.000000,0.299929
1,2.0,5.567290,10.590991,79.509127,20.643597,53.937462,0.063000,1.413690,7.218000,52.488000,...,3315.0,0,1265474.0,26417.0,88389.0,0.069847,2410.0,0.0,0.000000,0.091229
2,3.0,5.621669,92.770560,85.218453,19.863912,53.859214,0.059667,1.819006,7.364667,55.057667,...,5976.0,0,1189582.0,27430.0,90056.0,0.075704,9580.0,0.0,0.000000,0.349253
3,4.0,5.621901,84.495686,27.943097,20.734716,53.462761,0.067000,2.251050,7.208500,46.101000,...,3151.0,0,1229218.0,22179.0,63720.0,0.051838,7249.0,0.0,0.000000,0.326841
4,5.0,5.421339,94.876874,99.482576,21.357758,54.253761,0.052000,1.875000,7.375500,49.400500,...,3419.0,0,634855.0,22867.0,52195.0,0.082216,6156.0,0.0,0.000000,0.269209
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,277.0,1.997243,3.062395,98.907334,6.090387,50.649088,0.109000,3.666667,8.756000,83.165000,...,4209.0,0,85005.0,17441.0,45877.0,0.539698,2485.0,728.0,0.041741,0.142480
276,278.0,2.785375,4.405721,99.762551,6.289372,50.605630,0.138000,5.000000,7.850000,86.592000,...,799.0,0,55961.0,6898.0,17143.0,0.306338,865.0,99.0,0.014352,0.125399
277,279.0,3.585311,4.088966,99.687772,6.560337,50.494628,0.135000,2.500000,7.907000,81.355000,...,5686.0,0,119590.0,6860.0,17439.0,0.145823,995.0,0.0,0.000000,0.145044
278,280.0,3.452736,1.613553,70.916132,6.167253,50.364714,0.143000,2.785714,7.611000,93.091000,...,1548.0,0,276751.0,11907.0,29726.0,0.107411,3327.0,0.0,0.000000,0.279415


#### Public utility buildings

In [26]:
ifo_build_df = pd.read_csv(f"{ifo_path}\ipehd_1821_misc_buil.csv", encoding="ISO-8859-1")

In [27]:
ifo_build_df = ifo_build_df.groupby("kreiskey1800")["buil1821_pub"].sum().reset_index()

In [28]:
merged_df = pd.merge(merged_df, ifo_build_df, on="kreiskey1800", how="left")

In [29]:
merged_df["s_build"] = merged_df["buil1821_pub"]/merged_df["pop1821_tot"] * 1000
merged_df

,kreiskey1800,log_distance_carbon_c,s_prot,s_german,longitud,latitud,slope_km,soil_suitability,mean_temp,mean_preci,...,misc1821_area,underage,pop1821_tot,density,low_stud,high_stud,s_high_stud,s_low_stud,buil1821_pub,s_build
0,1.0,5.358620,95.806330,99.799868,20.548731,54.397921,0.050000,2.102802,7.367000,59.828000,...,1143405.0,25316.0,85831.0,0.075066,7593.0,0.0,0.000000,0.299929,284.0,3.308828
1,2.0,5.567290,10.590991,79.509127,20.643597,53.937462,0.063000,1.413690,7.218000,52.488000,...,1265474.0,26417.0,88389.0,0.069847,2410.0,0.0,0.000000,0.091229,158.0,1.787553
2,3.0,5.621669,92.770560,85.218453,19.863912,53.859214,0.059667,1.819006,7.364667,55.057667,...,1189582.0,27430.0,90056.0,0.075704,9580.0,0.0,0.000000,0.349253,325.0,3.608866
3,4.0,5.621901,84.495686,27.943097,20.734716,53.462761,0.067000,2.251050,7.208500,46.101000,...,1229218.0,22179.0,63720.0,0.051838,7249.0,0.0,0.000000,0.326841,348.0,5.461394
4,5.0,5.421339,94.876874,99.482576,21.357758,54.253761,0.052000,1.875000,7.375500,49.400500,...,634855.0,22867.0,52195.0,0.082216,6156.0,0.0,0.000000,0.269209,173.0,3.314494
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,277.0,1.997243,3.062395,98.907334,6.090387,50.649088,0.109000,3.666667,8.756000,83.165000,...,85005.0,17441.0,45877.0,0.539698,2485.0,728.0,0.041741,0.142480,50.0,1.089871
276,278.0,2.785375,4.405721,99.762551,6.289372,50.605630,0.138000,5.000000,7.850000,86.592000,...,55961.0,6898.0,17143.0,0.306338,865.0,99.0,0.014352,0.125399,30.0,1.749985
277,279.0,3.585311,4.088966,99.687772,6.560337,50.494628,0.135000,2.500000,7.907000,81.355000,...,119590.0,6860.0,17439.0,0.145823,995.0,0.0,0.000000,0.145044,44.0,2.523080
278,280.0,3.452736,1.613553,70.916132,6.167253,50.364714,0.143000,2.785714,7.611000,93.091000,...,276751.0,11907.0,29726.0,0.107411,3327.0,0.0,0.000000,0.279415,101.0,3.397699


#### Landownership concentration

In [30]:
ifo_agri_df = pd.read_csv(f"{ifo_path}\ipehd_1816_agri_land.csv", encoding="ISO-8859-1")
ifo_agri_df

,id1816,kreiskey1800,county,rb,land1816_far_ext_ov300,land1816_far_ext_15to30,land1816_far_ext_un15
0,1,277,Achen,AAC,7.0,881.0,2831.0
1,2,33,Adelnau,POS,100.0,1807.0,2027.0
2,3,254,Adenau,KOB,0.0,798.0,2805.0
3,4,196,Ahaus,MUN,5.0,197.0,3313.0
4,5,255,Ahrweiler,KOB,2.0,572.0,4385.0
...,...,...,...,...,...,...,...
303,304,276,Aachen,AAC,NaN,NaN,NaN
304,305,228,Köln,KOL,NaN,NaN,NaN
305,306,193,Münster,MUN,NaN,NaN,NaN
306,307,59,Potsdam,POT,0.0,14.0,48.0


In [31]:
ifo_agri_df = ifo_agri_df.groupby("kreiskey1800")[["land1816_far_ext_ov300", "land1816_far_ext_15to30", "land1816_far_ext_un15"]].sum().reset_index()

In [32]:
list_agri = ["land1816_far_ext_ov300", "land1816_far_ext_15to30", "land1816_far_ext_un15"]

ifo_agri_df["total_agri"] = 0

for col in list_agri:
    ifo_agri_df["total_agri"] = ifo_agri_df["total_agri"]+ifo_agri_df[col]

ifo_agri_df["s_agri"] = ifo_agri_df["land1816_far_ext_ov300"] / ifo_agri_df["total_agri"]
ifo_agri_df["s_agri"] = ifo_agri_df["s_agri"].astype(float)
ifo_agri_df

,kreiskey1800,land1816_far_ext_ov300,land1816_far_ext_15to30,land1816_far_ext_un15,total_agri,s_agri
0,1,241.0,2946.0,921.0,4108.0,0.058666
1,2,67.0,4629.0,1049.0,5745.0,0.011662
2,3,195.0,3274.0,729.0,4198.0,0.046451
3,4,105.0,5404.0,450.0,5959.0,0.017620
4,5,106.0,2463.0,477.0,3046.0,0.034800
...,...,...,...,...,...,...
275,277,7.0,881.0,2831.0,3719.0,0.001882
276,278,5.0,535.0,1525.0,2065.0,0.002421
277,279,0.0,318.0,2294.0,2612.0,0.000000
278,280,3.0,1128.0,3374.0,4505.0,0.000666


In [33]:
merged_df = pd.merge(merged_df, ifo_agri_df, on="kreiskey1800", how="left")

#### Water Mills

In [34]:
ifo_mill_df = pd.read_csv(f"{ifo_path}\ipehd_1819_indu_fac.csv", encoding="ISO-8859-1")
ifo_mill_df

,id1819,kreiskey1800,county,rb,fac1819_brick,fac1819_lime,fac1819_glass,mill1819_water,mill1819_saw_one,mill1819_saw_more
0,1,277,Achen,AAC,5,10,2,26,0,0
1,2,33,Adelnau,POS,11,6,0,26,1,0
2,3,254,Adenau,KOB,0,1,0,71,0,0
3,4,196,Ahaus,MUN,11,15,0,20,0,0
4,5,255,Ahrweiler,KOB,0,0,0,51,0,0
...,...,...,...,...,...,...,...,...,...,...
339,340,276,Aachen,AAC,10,1,1,10,0,0
340,341,228,Köln,KOL,0,0,1,4,0,0
341,342,193,Münster,MUN,0,0,0,2,0,0
342,343,59,Potsdam,POT,0,2,0,2,0,0


In [35]:
ifo_mill_df = ifo_mill_df.groupby("kreiskey1800")[["mill1819_water"]].sum().reset_index()

In [36]:
merged_df = pd.merge(merged_df, ifo_mill_df, on="kreiskey1800", how="left")

In [37]:
merged_df["s_mill"] = merged_df["mill1819_water"]/merged_df["pop1821_tot"]*1000

In [38]:
merged_df.columns

Index(['kreiskey1800', 'log_distance_carbon_c', 's_prot', 's_german',
       'longitud', 'latitud', 'slope_km', 'soil_suitability', 'mean_temp',
       'mean_preci', 'dist_100km', 'min_distance_uni', 'unskilled', 'low',
       'medium', 'high', 'd_Hauptberuf', 'steamengines_number', 'horsepower',
       'BER_dummy', 'misc1821_area', 'underage', 'pop1821_tot', 'density',
       'low_stud', 'high_stud', 's_high_stud', 's_low_stud', 'buil1821_pub',
       's_build', 'land1816_far_ext_ov300', 'land1816_far_ext_15to30',
       'land1816_far_ext_un15', 'total_agri', 's_agri', 'mill1819_water',
       's_mill'],
      dtype='object')

#### Skill shares

In [39]:
merged_df["per_unskilled"] = merged_df["unskilled"] / merged_df["d_Hauptberuf"]
merged_df["per_low"] = merged_df["low"] / merged_df["d_Hauptberuf"]
merged_df["per_medium"] = merged_df["medium"] / merged_df["d_Hauptberuf"]
merged_df["per_high"] = merged_df["high"] / merged_df["d_Hauptberuf"]

#### Check for NaNs

had to drop 11 observations

In [40]:
nan_columns = merged_df.isna().any()
nan_sum = merged_df.isna().sum()
print(nan_columns)
print(nan_sum)

kreiskey1800               False
log_distance_carbon_c      False
s_prot                     False
s_german                   False
longitud                   False
latitud                    False
slope_km                   False
soil_suitability           False
mean_temp                  False
mean_preci                 False
dist_100km                 False
min_distance_uni           False
unskilled                  False
low                        False
medium                     False
high                       False
d_Hauptberuf               False
steamengines_number        False
horsepower                 False
BER_dummy                  False
misc1821_area              False
underage                   False
pop1821_tot                False
density                    False
low_stud                   False
high_stud                  False
s_high_stud                 True
s_low_stud                  True
buil1821_pub               False
s_build                    False
land1816_f

In [41]:
merged_df = merged_df.dropna(subset=["s_high_stud"])
merged_df['s_high_stud'] = merged_df['s_high_stud'].replace([np.inf, -np.inf], 0)

In [42]:
merged_df = merged_df.dropna(subset=["s_low_stud"])
merged_df = merged_df.dropna(subset=["s_agri"])
#merged_df = merged_df[~merged_df["s_high_stud"].isin([np.inf, -np.inf])]
len(merged_df)

267

# Spatial Autocorrelation
First I check for spatial autocorrelation

In [43]:
gdf = gpd.GeoDataFrame(merged_df, geometry=gpd.points_from_xy(merged_df.longitud, merged_df.latitud))

In [44]:
# Compute the distance matrix
dist_matrix = distance_matrix(merged_df[["longitud", "latitud"]], merged_df[["longitud", "latitud"]])

# Compute the mean, median and 99th percentile distance
mean_distance = dist_matrix.mean()
median_distance = np.median(dist_matrix)  # Use numpy's median function as pd.median() operates on DataFrame/Series
percentile = np.percentile(dist_matrix, 99)

# Output
print(f"Mean Distance: {mean_distance}")
print(f"Median Distance: {median_distance}")
print(f"99th percentile: {percentile}")

Mean Distance: 5.4037625856801155
Median Distance: 4.8027921525902775
99th percentile: 14.526246265338697


In [45]:
# Creating spatial weights matrix (Euclidean distance)
w = DistanceBand.from_dataframe(gdf, threshold=median_distance, binary=False)

# Transform the matrix so all rows add up to 1
w.transform = "r"

C:\Users\soere\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\sparse\_data.py:117: RuntimeWarning: divide by zero encountered in reciprocal
  return self._with_data(data ** n)


In [46]:
merged_df.columns

Index(['kreiskey1800', 'log_distance_carbon_c', 's_prot', 's_german',
       'longitud', 'latitud', 'slope_km', 'soil_suitability', 'mean_temp',
       'mean_preci', 'dist_100km', 'min_distance_uni', 'unskilled', 'low',
       'medium', 'high', 'd_Hauptberuf', 'steamengines_number', 'horsepower',
       'BER_dummy', 'misc1821_area', 'underage', 'pop1821_tot', 'density',
       'low_stud', 'high_stud', 's_high_stud', 's_low_stud', 'buil1821_pub',
       's_build', 'land1816_far_ext_ov300', 'land1816_far_ext_15to30',
       'land1816_far_ext_un15', 'total_agri', 's_agri', 'mill1819_water',
       's_mill', 'per_unskilled', 'per_low', 'per_medium', 'per_high'],
      dtype='object')

In [47]:
# Check variables for spatial autocorrelation
mi_carbon = Moran(gdf["log_distance_carbon_c"], w)
mi_steam = Moran((gdf["steamengines_number"]/gdf["d_Hauptberuf"]), w)
mi_hp = Moran((gdf["horsepower"]/gdf["d_Hauptberuf"]), w)
mi_unskilled = Moran(gdf["per_unskilled"], w)
mi_low = Moran(gdf["per_low"], w)
mi_medium = Moran(gdf["per_medium"], w)
mi_high = Moran(gdf["per_high"], w)

# Output Moran's I value and p-value
print(f"log_distance_carbon_c - Moran's I: {mi_carbon.I}, p-value: {mi_carbon.p_sim}")
print(f"steamengines_number - Moran's I: {mi_steam.I}, p-value: {mi_steam.p_sim}")
print(f"horsepower - Moran's I: {mi_hp.I}, p-value: {mi_hp.p_sim}")
print(f"Unskilled - Moran's I: {mi_hp.I}, p-value: {mi_hp.p_sim}")
print(f"Low - Moran's I: {mi_hp.I}, p-value: {mi_hp.p_sim}")
print(f"Medium - Moran's I: {mi_hp.I}, p-value: {mi_hp.p_sim}")
print(f"High - Moran's I: {mi_hp.I}, p-value: {mi_hp.p_sim}")

log_distance_carbon_c - Moran's I: 0.3677952564115726, p-value: 0.001
steamengines_number - Moran's I: 0.11376551564719149, p-value: 0.001
horsepower - Moran's I: 0.1677225992623932, p-value: 0.001
Unskilled - Moran's I: 0.1677225992623932, p-value: 0.001
Low - Moran's I: 0.1677225992623932, p-value: 0.001
Medium - Moran's I: 0.1677225992623932, p-value: 0.001
High - Moran's I: 0.1677225992623932, p-value: 0.001


# 2SLS
Lets run a good'ol reliable IV first

In [48]:
# Transform steamengines to steamengines per worker
merged_df["steamengines_number"] = (merged_df["steamengines_number"] / merged_df["d_Hauptberuf"])*1000
merged_df["horsepower"] = (merged_df["horsepower"] / merged_df["d_Hauptberuf"])*1000

# Log some variables and add 1 if they contain zeros

merged_df["steamengines_number"] = np.log(merged_df["steamengines_number"])
merged_df["horsepower"] = np.log(merged_df["horsepower"])

In [49]:
merged_df.columns

Index(['kreiskey1800', 'log_distance_carbon_c', 's_prot', 's_german',
       'longitud', 'latitud', 'slope_km', 'soil_suitability', 'mean_temp',
       'mean_preci', 'dist_100km', 'min_distance_uni', 'unskilled', 'low',
       'medium', 'high', 'd_Hauptberuf', 'steamengines_number', 'horsepower',
       'BER_dummy', 'misc1821_area', 'underage', 'pop1821_tot', 'density',
       'low_stud', 'high_stud', 's_high_stud', 's_low_stud', 'buil1821_pub',
       's_build', 'land1816_far_ext_ov300', 'land1816_far_ext_15to30',
       'land1816_far_ext_un15', 'total_agri', 's_agri', 'mill1819_water',
       's_mill', 'per_unskilled', 'per_low', 'per_medium', 'per_high'],
      dtype='object')

## Try

In [50]:
merged_df["per_unskilled"] = merged_df["per_unskilled"]*100
merged_df["per_low"] = merged_df["per_low"]*100
merged_df["per_medium"] = merged_df["per_medium"]*100
merged_df["per_high"] = merged_df["per_high"]*100

In [51]:
merged_df["per_unskilled"] = np.log(merged_df["per_unskilled"])
merged_df["per_low"] = np.log(merged_df["per_low"])
merged_df["per_medium"] = np.log(merged_df["per_medium"])
merged_df["per_high"] = np.log(merged_df["per_high"])

## 2SLS function
I build a function that runs multiple 2SLS in a row, each time adding another variable from a list of control variables

In [52]:
def twoSLS(df, IV, first_stage_dependent, controls, second_stage_independent):
    """
    Performs a series of Two-Stage Least Squares (2SLS) regressions on a given DataFrame.
    In each iteration, an additional control variable from the provided list is included.

    Parameters:
    df (DataFrame): The DataFrame containing the data for regression.
    IV (str): The name of the instrumental variable column in the DataFrame.
    first_stage_dependent (str): The name of the dependent variable for the first stage regression.
    controls (list of str): A list of control variable names. Each iteration includes an additional control variable.
    second_stage_independent (str): The name of the main independent variable for the second stage regression.

    Returns:
    tuple of DataFrames: The first DataFrame contains the results (coefficients, significance, 
                         Adj. R-squared, F-statistic) of each first stage regression. 
                         The second DataFrame contains the results 
                         of each second stage regression. Significance is denoted by stars: 
                         *** for p < 0.01, ** for p < 0.05, and * for p < 0.1.
                         The third DataFrame contains the residuals.
    """

    first_stage_results = []
    second_stage_results = []
    # For analysis of spatial autocorrelation
    residuals = []

    for i in range(len(controls) + 1):
        # Subset of control variables for this iteration
        subset_controls = controls[:i]

        # Build the formula, handling the case with no controls
        controls_formula_part = " + ".join(subset_controls)
        if controls_formula_part:
            controls_formula_part = " + " + controls_formula_part

        # To avoid SettingWithCopyWarning
        df_copy = df.copy()

        # First Stage Regression
        first_stage_formula = f"{first_stage_dependent} ~ {IV}{controls_formula_part}"
        first_stage_model = smf.ols(first_stage_formula, data=df_copy)
        first_stage_fit = first_stage_model.fit()

        # Add the predicted values as a new column in the copy
        df_copy["predicted"] = first_stage_fit.predict(df_copy)

        # Second Stage Regression
        second_stage_formula = f"{second_stage_independent} ~ predicted{controls_formula_part}"
        second_stage_model = smf.ols(second_stage_formula, data=df_copy)
        second_stage_fit = second_stage_model.fit()

        def format_coef(coef, std_err, p_value):
            stars = ""
            if p_value < 0.01:
                stars = "***"
            elif p_value < 0.05:
                stars = "**"
            elif p_value < 0.1:
                stars = "*"
    
            # Check if coef or std_err is smaller than 0.000 and format accordingly
            coef_str = f"{coef:.10e}" if abs(coef) < 0.000 else f"{coef:.3f}"
            std_err_str = f"{std_err:.10e}" if abs(std_err) < 0.000 else f"{std_err:.3f}"

            return f"{coef_str}{stars} ({std_err_str})"


        # Extract coefficients and significance of first stage
        first_stage_summary = first_stage_fit.summary2().tables[1]
        first_stage_coefs = first_stage_summary.apply(lambda row: format_coef(row["Coef."], row["Std.Err."], row["P>|t|"]), axis=1)

        # Extract coefficients and significance of second stage
        second_stage_summary = second_stage_fit.summary2().tables[1]
        second_stage_coefs = second_stage_summary.apply(lambda row: format_coef(row["Coef."], row["Std.Err."], row["P>|t|"]), axis=1)

        # Adding R-squared and F-statistic
        first_stage_coefs.loc["Adj. R-squared"] = f"{first_stage_fit.rsquared_adj:.3f}"
        first_stage_coefs.loc["F-statistic"] = f"{first_stage_fit.fvalue:.3f}"
        second_stage_coefs.loc["Adj. R-squared"] = f"{second_stage_fit.rsquared_adj:.3f}"
        
        # Adding Standard Errors
        second_stage_se = second_stage_fit.bse["predicted"]
        second_stage_coefs.loc["std err"] = second_stage_se
        
        # Residuals
        res = second_stage_fit.resid
        
        # Append 
        first_stage_results.append(first_stage_coefs)
        second_stage_results.append(second_stage_coefs)
        residuals.append(res)

    # Convert the results to DataFrame
    first_stage_df = pd.DataFrame(first_stage_results).T
    second_stage_df = pd.DataFrame(second_stage_results).T
    
    # Df of residuals
    residuals_df = pd.DataFrame(residuals).T

    return first_stage_df, second_stage_df, residuals_df

## Use the Function

In [53]:
# DataFrame that contains all the variables for the regression analysis
dep_var = merged_df[["log_distance_carbon_c", 
                     "s_prot", 
                     "s_german", 
                     "longitud", 
                     "latitud", 
                     "soil_suitability", 
                     "mean_temp", 
                     "mean_preci", 
                     "dist_100km", 
                     "min_distance_uni",  
                     "BER_dummy", 
                     "density",
                     "steamengines_number",
                     "per_low",
                     "per_medium",
                     "per_unskilled",
                     "per_high",
                     "d_Hauptberuf",
                     "s_low_stud",
                     "s_high_stud",
                     "s_build",
                     "s_agri",
                     "s_mill"]]

In [54]:
# Make two lists of exogenous variables
# Exclude latitude and longitude once
control_list = ["longitud",
                "latitud", 
                "soil_suitability", 
                "mean_temp", 
                "mean_preci", 
                "dist_100km",
                "min_distance_uni",
                 "s_prot", 
                "s_german",
                "density",
                "s_mill",
                "s_build",
                "s_low_stud",
                "s_high_stud",
                "s_agri"]

control_list_2 = ["s_prot", 
                  "s_german", 
                  "soil_suitability", 
                  "mean_temp", 
                  "mean_preci", 
                  "dist_100km",
                  "min_distance_uni",   
                  "density",
                  "s_low_stud",
                  "s_high_stud",
                  "s_build",
                  "s_agri",
                  "s_mill"]

In [55]:
first_unskilled, second_unskilled, res_unskilled = twoSLS(dep_var, "log_distance_carbon_c", "steamengines_number", control_list, "per_unskilled")
first_low, second_low, res_low = twoSLS(dep_var, "log_distance_carbon_c", "steamengines_number", control_list, "per_low")
first_medium, second_medium, res_medium = twoSLS(dep_var, "log_distance_carbon_c", "steamengines_number", control_list, "per_medium")
first_high, second_high, res_high = twoSLS(dep_var, "log_distance_carbon_c", "steamengines_number", control_list, "per_high")

In [56]:
first_unskilled[[1,15]]

,1,15
Intercept,2.528*** (0.102),-6.004*** (2.131)
log_distance_carbon_c,-0.087*** (0.023),-0.176*** (0.027)
Adj. R-squared,0.044,0.352
F-statistic,7.110,10.043
longitud,0.016** (0.008),0.029** (0.014)
latitud,NaN,0.123*** (0.041)
soil_suitability,NaN,-0.115*** (0.034)
mean_temp,NaN,0.274*** (0.057)
mean_preci,NaN,0.008** (0.004)
dist_100km,NaN,-0.025 (0.136)


In [57]:
# First Stage (same for every regression)
second_unskilled[[1, 15]]

,1,15
Intercept,4.601*** (0.278),4.466*** (0.768)
predicted,-0.387*** (0.115),-0.122** (0.062)
Adj. R-squared,0.151,0.501
std err,0.114557,0.06165
longitud,0.019*** (0.003),-0.013** (0.006)
latitud,NaN,0.023 (0.014)
soil_suitability,NaN,-0.006 (0.016)
mean_temp,NaN,-0.088*** (0.030)
mean_preci,NaN,-0.007*** (0.002)
dist_100km,NaN,0.046 (0.055)


In [58]:
second_low[[1, 15]]

,1,15
Intercept,1.711*** (0.345),4.353*** (1.060)
predicted,0.760*** (0.143),0.169** (0.085)
Adj. R-squared,0.293,0.489
std err,0.142562,0.08507
longitud,-0.036*** (0.004),0.016* (0.008)
latitud,NaN,-0.063*** (0.019)
soil_suitability,NaN,0.044* (0.023)
mean_temp,NaN,0.080* (0.041)
mean_preci,NaN,0.009*** (0.002)
dist_100km,NaN,-0.096 (0.075)


In [59]:
second_medium[[1, 15]]

,1,15
Intercept,2.891*** (0.227),2.168*** (0.654)
predicted,0.154 (0.094),0.112** (0.053)
Adj. R-squared,0.018,0.372
std err,0.093561,0.052503
longitud,-0.005** (0.002),0.002 (0.005)
latitud,NaN,0.007 (0.012)
soil_suitability,NaN,-0.016 (0.014)
mean_temp,NaN,0.021 (0.025)
mean_preci,NaN,0.002 (0.001)
dist_100km,NaN,-0.020 (0.046)


In [60]:
second_high[[1, 15]]

,1,15
Intercept,-0.698 (0.563),-2.076 (1.722)
predicted,0.256 (0.232),0.229* (0.138)
Adj. R-squared,0.023,0.298
std err,0.232279,0.138181
longitud,-0.017*** (0.006),0.003 (0.014)
latitud,NaN,0.009 (0.031)
soil_suitability,NaN,0.012 (0.037)
mean_temp,NaN,0.039 (0.066)
mean_preci,NaN,0.004 (0.004)
dist_100km,NaN,-0.218* (0.122)


# Check error terms for spatial autocorrelation
Use the residuals

## Unskilled

In [61]:
test_statistics = []
p_values = []

for column in res_unskilled.columns:
    mi_res = Moran(res_unskilled[column], w)
    
    test_statistics.append(mi_res.I)
    p_values.append(mi_res.p_sim)

for i in range(len(test_statistics)):
    
    print(f"Variable {i+1}: Test Statistic = {test_statistics[i]}, P-value = {p_values[i]}")

Variable 1: Test Statistic = 0.0647352856347792, P-value = 0.001
Variable 2: Test Statistic = 0.01925233368336648, P-value = 0.025
Variable 3: Test Statistic = 0.014258260082433636, P-value = 0.042
Variable 4: Test Statistic = 0.01145751589002968, P-value = 0.065
Variable 5: Test Statistic = 0.012160648292362101, P-value = 0.059
Variable 6: Test Statistic = 0.012063352470554673, P-value = 0.051
Variable 7: Test Statistic = 0.010895243258068723, P-value = 0.069
Variable 8: Test Statistic = 0.008506489805273706, P-value = 0.081
Variable 9: Test Statistic = 0.0035183075012925743, P-value = 0.203
Variable 10: Test Statistic = 0.0037459059788278915, P-value = 0.19
Variable 11: Test Statistic = -0.022001040090184804, P-value = 0.009
Variable 12: Test Statistic = -0.02233774659571709, P-value = 0.009
Variable 13: Test Statistic = -0.02086323571314059, P-value = 0.019
Variable 14: Test Statistic = -0.021229034252131917, P-value = 0.012
Variable 15: Test Statistic = -0.021034679934737538, P-val

## Low

In [62]:
test_statistics = []
p_values = []

for column in res_low.columns:
    mi_res = Moran(res_unskilled[column], w)
    
    test_statistics.append(mi_res.I)
    p_values.append(mi_res.p_sim)

for i in range(len(test_statistics)):
    
    print(f"Variable {i+1}: Test Statistic = {test_statistics[i]}, P-value = {p_values[i]}")

Variable 1: Test Statistic = 0.0647352856347792, P-value = 0.001
Variable 2: Test Statistic = 0.01925233368336648, P-value = 0.02
Variable 3: Test Statistic = 0.014258260082433636, P-value = 0.044
Variable 4: Test Statistic = 0.01145751589002968, P-value = 0.076
Variable 5: Test Statistic = 0.012160648292362101, P-value = 0.068
Variable 6: Test Statistic = 0.012063352470554673, P-value = 0.046
Variable 7: Test Statistic = 0.010895243258068723, P-value = 0.056
Variable 8: Test Statistic = 0.008506489805273706, P-value = 0.081
Variable 9: Test Statistic = 0.0035183075012925743, P-value = 0.184
Variable 10: Test Statistic = 0.0037459059788278915, P-value = 0.176
Variable 11: Test Statistic = -0.022001040090184804, P-value = 0.008
Variable 12: Test Statistic = -0.02233774659571709, P-value = 0.011
Variable 13: Test Statistic = -0.02086323571314059, P-value = 0.014
Variable 14: Test Statistic = -0.021229034252131917, P-value = 0.011
Variable 15: Test Statistic = -0.021034679934737538, P-val

## Medium

In [63]:
test_statistics = []
p_values = []

for column in res_low.columns:
    mi_res = Moran(res_unskilled[column], w)
    
    test_statistics.append(mi_res.I)
    p_values.append(mi_res.p_sim)

for i in range(len(test_statistics)):
    
    print(f"Variable {i+1}: Test Statistic = {test_statistics[i]}, P-value = {p_values[i]}")

Variable 1: Test Statistic = 0.0647352856347792, P-value = 0.001
Variable 2: Test Statistic = 0.01925233368336648, P-value = 0.025
Variable 3: Test Statistic = 0.014258260082433636, P-value = 0.047
Variable 4: Test Statistic = 0.01145751589002968, P-value = 0.073
Variable 5: Test Statistic = 0.012160648292362101, P-value = 0.062
Variable 6: Test Statistic = 0.012063352470554673, P-value = 0.078
Variable 7: Test Statistic = 0.010895243258068723, P-value = 0.057
Variable 8: Test Statistic = 0.008506489805273706, P-value = 0.108
Variable 9: Test Statistic = 0.0035183075012925743, P-value = 0.181
Variable 10: Test Statistic = 0.0037459059788278915, P-value = 0.179
Variable 11: Test Statistic = -0.022001040090184804, P-value = 0.007
Variable 12: Test Statistic = -0.02233774659571709, P-value = 0.006
Variable 13: Test Statistic = -0.02086323571314059, P-value = 0.022
Variable 14: Test Statistic = -0.021229034252131917, P-value = 0.01
Variable 15: Test Statistic = -0.021034679934737538, P-val

## High

In [64]:
test_statistics = []
p_values = []

for column in res_low.columns:
    mi_res = Moran(res_unskilled[column], w)
    
    test_statistics.append(mi_res.I)
    p_values.append(mi_res.p_sim)

for i in range(len(test_statistics)):
    
    print(f"Variable {i+1}: Test Statistic = {test_statistics[i]}, P-value = {p_values[i]}")

Variable 1: Test Statistic = 0.0647352856347792, P-value = 0.001
Variable 2: Test Statistic = 0.01925233368336648, P-value = 0.015
Variable 3: Test Statistic = 0.014258260082433636, P-value = 0.052
Variable 4: Test Statistic = 0.01145751589002968, P-value = 0.058
Variable 5: Test Statistic = 0.012160648292362101, P-value = 0.068
Variable 6: Test Statistic = 0.012063352470554673, P-value = 0.051
Variable 7: Test Statistic = 0.010895243258068723, P-value = 0.078
Variable 8: Test Statistic = 0.008506489805273706, P-value = 0.109
Variable 9: Test Statistic = 0.0035183075012925743, P-value = 0.182
Variable 10: Test Statistic = 0.0037459059788278915, P-value = 0.179
Variable 11: Test Statistic = -0.022001040090184804, P-value = 0.01
Variable 12: Test Statistic = -0.02233774659571709, P-value = 0.008
Variable 13: Test Statistic = -0.02086323571314059, P-value = 0.01
Variable 14: Test Statistic = -0.021229034252131917, P-value = 0.01
Variable 15: Test Statistic = -0.021034679934737538, P-value

# Control for spatial autocorrelation

## IV with lag
In the second stage, lag controls on the dependent variable (i.e. skill share)

### Function

In [65]:
def IV_w_lag(df, IV, first_stage_dependent, dependent_var, controls, w):
    """
    Performs a series of IV regressions with spatial lag in the second-stage,
    incorporating a first-stage regression similar to twoSLS.

    Parameters:
    df (DataFrame): The DataFrame containing the data for regression.
    IV (str): The name of the instrumental variable column in the DataFrame.
    first_stage_dependent (str): The name of the dependent variable for the first stage regression.
    dependent_var (str): The name of the dependent variable for the regression.
    controls (list of str): A list of control variable names for both stages.
    w (libpysal.weights): weight matrix to control for spatial autocorrelation.

    Returns:
    Tuple of DataFrames
    First DataFrame containing the results of each second stage regression.
    Second DataFrame containing the residuals.
    Third DataFrame containing the pseudo-Rsquares.
    Fourth DataFrame containing the p-values from the AK test.
    """

    results = []
    res = []
    # Pseudo R-Squared
    pseudo_R = []
    # AKtest
    ak_pvalues = []

    for i in range(len(controls) + 1):
        subset_controls = controls[:i]

        # First Stage Regression
        df_copy = df.copy()
        controls_formula_part = " + ".join(subset_controls)
        first_stage_formula = f"{first_stage_dependent} ~ {IV}{(' + ' if controls_formula_part else '')}{controls_formula_part}"
        first_stage_model = smf.ols(first_stage_formula, data=df_copy)
        first_stage_fit = first_stage_model.fit()
        df_copy["predicted"] = first_stage_fit.predict(df_copy)

        # Second Stage Regression (Spatial Lag)
        X = df_copy[["predicted"] + subset_controls]
        y = df_copy[dependent_var]
        slm = spreg.ML_Lag(y.values, X.values, w, slx_lags = 0, name_y=dependent_var, name_x=["predicted"] + subset_controls)

        # Formatting results
        p_value = slm.output["prob"]
        coefs = slm.output["coefficients"]
        names = slm.output["var_names"]
        r = slm.pr2
        residuals = slm.e_pred
        
        # AKtest
        ak = AKtest(slm, w)
        ak_pvalues.append(ak.p)
        
        results.append(names)
        results.append(coefs)
        results.append(p_value)
        res.append(list(residuals))
        pseudo_R.append(r)

    # Convert the results to DataFrame
    results_df = pd.DataFrame(results).T
    residuals_df = pd.DataFrame(res).T
    residuals_df = residuals_df.astype(float)
    pseudoR_df = pd.DataFrame(pseudo_R)
    ak_pvalues_df = pd.DataFrame(ak_pvalues)

    return results_df, residuals_df, pseudoR_df, ak_pvalues_df

### Run the Function

In [66]:
lag_unskilled_second, res_lag_unskilled, R_un, ak_un = IV_w_lag(dep_var, "log_distance_carbon_c", "steamengines_number",
                                                          "per_unskilled", control_list, w)
lag_low_second, res_lag_low, R_low, ak_low = IV_w_lag(dep_var, "log_distance_carbon_c", "steamengines_number",
                                              "per_low", control_list, w)
lag_medium_second, res_lag_medium, R_medium, ak_medium = IV_w_lag(dep_var, "log_distance_carbon_c", "steamengines_number",
                                                       "per_medium", control_list, w)
lag_high_second, res_lag_high, R_high, ak_high = IV_w_lag(dep_var, "log_distance_carbon_c", "steamengines_number",
                                                 "per_high", control_list, w)

In [67]:
pd.set_option('display.max_columns', 40)

#### Unskilled

In [68]:
lag_unskilled_second

,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,...,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob
0,CONSTANT,2.59454,0.000805,CONSTANT,2.937254,0.002004,CONSTANT,1.327583,0.191104,CONSTANT,1.351365,0.190859,CONSTANT,2.148385,0.058031,CONSTANT,3.029759,0.018649,CONSTANT,3.089698,...,4.573749,0.001251,CONSTANT,5.270004,0.000196,CONSTANT,5.35092,0.00028,CONSTANT,5.140475,0.000456,CONSTANT,4.759452,0.001289,CONSTANT,5.246076,0.000166,CONSTANT,5.109757,0.00035
1,predicted,-0.500742,0.000439,predicted,-0.358082,0.00193,predicted,-0.207769,0.000955,predicted,-0.233314,0.00052,predicted,-0.245059,0.000113,predicted,-0.236867,0.000239,predicted,-0.246163,...,-0.163495,0.028102,predicted,-0.147637,0.021873,predicted,-0.148135,0.026646,predicted,-0.148271,0.024684,predicted,-0.154623,0.017564,predicted,-0.127054,0.037085,predicted,-0.126502,0.035529
2,W_per_unskilled,0.647747,0.000011,longitud,0.011535,0.017544,longitud,0.007921,0.123602,longitud,0.005874,0.270665,longitud,0.005255,0.348633,longitud,-0.000878,0.901131,longitud,0.000468,...,-0.010702,0.198928,longitud,-0.003306,0.678629,longitud,-0.00275,0.729233,longitud,-0.002826,0.717037,longitud,-0.004656,0.552972,longitud,-0.010247,0.174069,longitud,-0.010717,0.153018
3,NaN,NaN,NaN,W_per_unskilled,0.433559,0.060921,latitud,0.027521,0.035844,latitud,0.032808,0.015921,latitud,0.035476,0.006194,latitud,0.034251,0.007942,latitud,0.033236,...,0.034125,0.008405,latitud,0.031665,0.005267,latitud,0.032423,0.012223,latitud,0.032424,0.01222,latitud,0.035296,0.007239,latitud,0.022423,0.071088,latitud,0.022769,0.091138
4,NaN,NaN,NaN,NaN,NaN,NaN,W_per_unskilled,0.399184,0.096742,soil_suitability,-0.045355,0.014507,soil_suitability,-0.046361,0.010388,soil_suitability,-0.039332,0.038754,soil_suitability,-0.040987,...,-0.024166,0.223733,soil_suitability,0.004638,0.783918,soil_suitability,0.004598,0.789326,soil_suitability,0.004582,0.788882,soil_suitability,0.002514,0.882458,soil_suitability,-0.006232,0.697567,soil_suitability,-0.006241,0.694717
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,W_per_unskilled,0.372235,0.133362,mean_temp,-0.031475,0.204141,mean_temp,-0.058836,0.063672,mean_temp,-0.042396,...,-0.092584,0.010136,mean_temp,-0.075941,0.018684,mean_temp,-0.075532,0.015869,mean_temp,-0.074103,0.016967,mean_temp,-0.073187,0.018097,mean_temp,-0.088697,0.002079,mean_temp,-0.088601,0.002089
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,W_per_unskilled,0.212241,0.444408,mean_preci,-0.002796,0.097029,mean_preci,-0.002878,...,-0.00537,0.006683,mean_preci,-0.005648,0.001118,mean_preci,-0.005662,0.001013,mean_preci,-0.005399,0.001742,mean_preci,-0.005423,0.001659,mean_preci,-0.006732,0.00003,mean_preci,-0.006725,0.000038
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,W_per_unskilled,0.114642,0.699404,dist_100km,0.103146,...,0.059149,0.356397,dist_100km,0.039427,0.483976,dist_100km,0.039109,0.489865,dist_100km,0.041389,0.464135,dist_100km,0.044601,0.429026,dist_100km,0.045412,0.389218,dist_100km,0.045646,0.386538
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,W_per_unskilled,0.073437,...,0.078522,0.006973,min_distance_uni,0.044444,0.096041,min_distance_uni,0.044701,0.091281,min_distance_uni,0.046665,0.084866,min_distance_uni,0.046684,0.084193,min_distance_uni,0.03672,0.147643,min_distance_uni,0.036012,0.155926
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.000292,0.612833,s_prot,-0.000304,0.549651,s_prot,-0.00031,0.548972,s_prot,-0.000394,0.481095,s_prot,-0.000483,0.394762,s_prot,-0.000067,0.900533,s_prot,-0.000064,0.90503


In [69]:
R_un

,0
0,0.168504
1,0.166398
2,0.170830
3,0.175110
4,0.230129
5,0.253675
6,0.260634
7,0.281281
8,0.293714
9,0.296774


In [70]:
ak_un

,0
0,0.601927
1,0.439075
2,0.609549
3,0.742958
4,0.473366
5,0.371744
6,0.385549
7,0.317659
8,0.531188
9,0.543614


#### Low

In [71]:
lag_low_second

,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,...,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob
0,CONSTANT,-1.328367,0.000932,CONSTANT,-0.710719,0.125289,CONSTANT,3.99623,0.000046,CONSTANT,4.304981,0.000031,CONSTANT,4.240627,0.000052,CONSTANT,4.785046,0.000288,CONSTANT,5.018784,...,4.405963,0.00124,CONSTANT,4.345869,0.001084,CONSTANT,3.862131,0.006578,CONSTANT,4.30832,0.00356,CONSTANT,4.340455,0.00275,CONSTANT,2.62738,0.04632,CONSTANT,2.38494,0.078892
1,predicted,0.808034,0.000003,predicted,0.611305,0.000009,predicted,0.220719,0.003282,predicted,0.260443,0.001102,predicted,0.259801,0.000695,predicted,0.23971,0.002171,predicted,0.249322,...,0.192261,0.034439,predicted,0.175244,0.042574,predicted,0.181948,0.042941,predicted,0.188612,0.032979,predicted,0.200128,0.022204,predicted,0.168604,0.041581,predicted,0.163103,0.046019
2,W_per_low,0.795625,0.0,longitud,-0.009824,0.039914,longitud,-0.00032,0.950267,longitud,0.002669,0.626762,longitud,0.002841,0.612466,longitud,0.009595,0.235633,longitud,0.008254,...,0.022228,0.020701,longitud,0.019828,0.036741,longitud,0.020359,0.030755,longitud,0.016844,0.074516,longitud,0.018781,0.046789,longitud,0.026362,0.003145,longitud,0.025932,0.003902
3,NaN,NaN,NaN,W_per_low,0.79102,0.0,latitud,-0.072066,0.000009,latitud,-0.081605,0.000001,latitud,-0.080908,0.000001,latitud,-0.087799,0.0,latitud,-0.087043,...,-0.0845,0.000001,latitud,-0.081331,0.000001,latitud,-0.073777,0.000065,latitud,-0.076198,0.000044,latitud,-0.079808,0.000021,latitud,-0.061166,0.000566,latitud,-0.053391,0.005051
4,NaN,NaN,NaN,NaN,NaN,NaN,W_per_low,0.743165,0.0,soil_suitability,0.073076,0.000989,soil_suitability,0.072884,0.000975,soil_suitability,0.06154,0.008015,soil_suitability,0.063313,...,0.051157,0.036424,soil_suitability,0.029284,0.199385,soil_suitability,0.030669,0.187422,soil_suitability,0.031821,0.167244,soil_suitability,0.035051,0.126184,soil_suitability,0.045378,0.03838,soil_suitability,0.043321,0.046278
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,W_per_low,0.705339,0.000001,mean_temp,-0.002603,0.931498,mean_temp,0.049882,0.205196,mean_temp,0.029368,...,0.067116,0.132651,mean_temp,0.054054,0.218811,mean_temp,0.059913,0.160557,mean_temp,0.056429,0.180793,mean_temp,0.054711,0.192814,mean_temp,0.071095,0.072222,mean_temp,0.071742,0.069552
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,W_per_low,0.721648,0.0,mean_preci,0.00557,0.011728,mean_preci,0.005768,...,0.007019,0.005402,mean_preci,0.007053,0.003616,mean_preci,0.007093,0.003305,mean_preci,0.006438,0.007283,mean_preci,0.006436,0.007188,mean_preci,0.007695,0.000619,mean_preci,0.007531,0.000968
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,W_per_low,0.409353,0.06063,dist_100km,-0.13403,...,-0.094189,0.234407,dist_100km,-0.078968,0.301666,dist_100km,-0.084296,0.273834,dist_100km,-0.093585,0.221447,dist_100km,-0.097975,0.198865,dist_100km,-0.097524,0.177482,dist_100km,-0.093168,0.197333
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,W_per_low,0.382147,...,-0.05704,0.106737,min_distance_uni,-0.029116,0.410943,min_distance_uni,-0.032763,0.350942,min_distance_uni,-0.043486,0.227846,min_distance_uni,-0.04427,0.218243,min_distance_uni,-0.031683,0.353439,min_distance_uni,-0.037194,0.276563
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.000494,0.494692,s_prot,-0.000483,0.488286,s_prot,-0.000531,0.455405,s_prot,-0.000252,0.741553,s_prot,-0.000106,0.891401,s_prot,-0.000545,0.462868,s_prot,-0.000484,0.511662


In [72]:
R_low

,0
0,0.352819
1,0.352567
2,0.354279
3,0.365471
4,0.379513
5,0.401409
6,0.407573
7,0.415305
8,0.417290
9,0.423420


In [73]:
ak_low

,0
0,0.124126
1,0.120337
2,0.126835
3,0.299571
4,0.161333
5,0.242020
6,0.247068
7,0.254103
8,0.320030
9,0.378861


#### Medium

In [74]:
lag_medium_second

,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,...,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob
0,CONSTANT,-0.075268,0.789401,CONSTANT,-0.061273,0.814787,CONSTANT,-0.045513,0.928515,CONSTANT,-0.025566,0.960043,CONSTANT,0.017115,0.975792,CONSTANT,-0.114423,0.853981,CONSTANT,-0.045955,...,-0.13935,0.83948,CONSTANT,-0.044213,0.948098,CONSTANT,0.848721,0.257358,CONSTANT,0.794116,0.277428,CONSTANT,0.790765,0.277239,CONSTANT,0.260703,0.717563,CONSTANT,0.313725,0.672127
1,predicted,0.154144,0.123463,predicted,0.141931,0.093942,predicted,0.165395,0.000323,predicted,0.168089,0.00064,predicted,0.174324,0.000206,predicted,0.170124,0.000456,predicted,0.175465,...,0.109068,0.050709,predicted,0.09912,0.059654,predicted,0.089975,0.095382,predicted,0.08945,0.092698,predicted,0.09044,0.086617,predicted,0.075584,0.135509,predicted,0.076376,0.126693
2,W_per_medium,0.905238,0.0,longitud,-0.000556,0.804186,longitud,-0.001616,0.540584,longitud,-0.001425,0.610234,longitud,-0.000519,0.864113,longitud,0.000226,0.961701,longitud,-0.000271,...,0.011004,0.044505,longitud,0.008839,0.099927,longitud,0.007583,0.147987,longitud,0.007376,0.15322,longitud,0.007644,0.143063,longitud,0.009946,0.049935,longitud,0.009931,0.049486
3,NaN,NaN,NaN,W_per_medium,0.912357,0.0,latitud,0.00404,0.673015,latitud,0.003635,0.714042,latitud,0.003593,0.704316,latitud,0.003185,0.738351,latitud,0.004055,...,0.002897,0.762572,latitud,0.004641,0.612828,latitud,-0.007577,0.463894,latitud,-0.008056,0.434689,latitud,-0.008746,0.408152,latitud,-0.000519,0.959866,latitud,-0.001841,0.870082
4,NaN,NaN,NaN,NaN,NaN,NaN,W_per_medium,0.828478,0.0,soil_suitability,0.004161,0.759837,soil_suitability,0.00424,0.75211,soil_suitability,0.00358,0.802605,soil_suitability,0.004495,...,-0.008677,0.561002,soil_suitability,-0.023006,0.095471,soil_suitability,-0.025139,0.069585,soil_suitability,-0.025091,0.068398,soil_suitability,-0.024691,0.072514,soil_suitability,-0.018769,0.158561,soil_suitability,-0.018449,0.162326
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,W_per_medium,0.823,0.0,mean_temp,0.012164,0.508276,mean_temp,0.013927,0.555552,mean_temp,0.002682,...,0.041383,0.126586,mean_temp,0.032765,0.216045,mean_temp,0.022459,0.374663,mean_temp,0.021811,0.382545,mean_temp,0.021846,0.380832,mean_temp,0.031762,0.184512,mean_temp,0.031787,0.183721
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,W_per_medium,0.769136,0.0,mean_preci,0.000158,0.89858,mean_preci,0.0002,...,0.002045,0.168144,mean_preci,0.002097,0.140199,mean_preci,0.001976,0.156772,mean_preci,0.001805,0.192075,mean_preci,0.001832,0.186493,mean_preci,0.002696,0.043892,mean_preci,0.002737,0.043936
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,W_per_medium,0.809981,0.0,dist_100km,-0.070152,...,-0.040283,0.404343,dist_100km,-0.03034,0.511494,dist_100km,-0.021493,0.639024,dist_100km,-0.023834,0.602117,dist_100km,-0.024513,0.591843,dist_100km,-0.024943,0.569712,dist_100km,-0.025662,0.558327
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,W_per_medium,0.806787,...,-0.04895,0.023266,min_distance_uni,-0.030973,0.147656,min_distance_uni,-0.025427,0.224237,min_distance_uni,-0.027506,0.19952,min_distance_uni,-0.027584,0.198618,min_distance_uni,-0.02238,0.278578,min_distance_uni,-0.021624,0.294373
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000341,0.4354,s_prot,0.000346,0.409364,s_prot,0.000441,0.295897,s_prot,0.000509,0.261324,s_prot,0.000523,0.256805,s_prot,0.000259,0.5642,s_prot,0.000247,0.579495


In [75]:
R_medium

,0
0,0.215887
1,0.223416
2,0.228090
3,0.231565
4,0.261619
5,0.268157
6,0.273498
7,0.286282
8,0.300646
9,0.308816


In [76]:
ak_medium

,0
0,0.001943
1,0.002389
2,0.340290
3,0.324451
4,0.568853
5,0.429028
6,0.389376
7,0.390248
8,0.686767
9,0.636665


#### High

In [77]:
lag_high_second

,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,...,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob
0,CONSTANT,-0.85378,0.166853,CONSTANT,-0.659278,0.206453,CONSTANT,-0.794211,0.487832,CONSTANT,-0.746521,0.516391,CONSTANT,-1.226822,0.315583,CONSTANT,-1.082732,0.445614,CONSTANT,-0.84982,...,-1.89401,0.178211,CONSTANT,-1.819932,0.186115,CONSTANT,-0.887997,0.575935,CONSTANT,-1.090014,0.494519,CONSTANT,-0.660953,0.680952,CONSTANT,-2.296425,0.142121,CONSTANT,-1.484608,0.364943
1,predicted,0.337073,0.184077,predicted,0.27353,0.203732,predicted,0.284079,0.01604,predicted,0.300373,0.017626,predicted,0.311927,0.009597,predicted,0.311838,0.012674,predicted,0.335883,...,0.216137,0.13553,predicted,0.199117,0.152416,predicted,0.186222,0.196247,predicted,0.183522,0.199149,predicted,0.209005,0.138019,predicted,0.164936,0.219392,predicted,0.177334,0.178899
2,W_per_high,0.872578,0.0,longitud,-0.003052,0.595798,longitud,-0.004212,0.537592,longitud,-0.002541,0.725229,longitud,0.003222,0.678923,longitud,0.002,0.868593,longitud,-0.000644,...,0.019102,0.178015,longitud,0.013112,0.355896,longitud,0.012074,0.38799,longitud,0.013285,0.343573,longitud,0.016071,0.253558,longitud,0.022941,0.092236,longitud,0.022421,0.096739
3,NaN,NaN,NaN,W_per_high,0.876546,0.0,latitud,0.002095,0.932019,latitud,-0.001343,0.958007,latitud,-0.004045,0.867966,latitud,-0.004936,0.841023,latitud,-0.001058,...,0.005295,0.83176,latitud,0.009634,0.691172,latitud,-0.004926,0.858706,latitud,-0.003211,0.907727,latitud,-0.013201,0.640266,latitud,0.009675,0.723587,latitud,-0.008186,0.78263
4,NaN,NaN,NaN,NaN,NaN,NaN,W_per_high,0.822643,0.0,soil_suitability,0.030221,0.389362,soil_suitability,0.029549,0.393543,soil_suitability,0.030979,0.403024,soil_suitability,0.035175,...,0.009122,0.81448,soil_suitability,-0.019473,0.59479,soil_suitability,-0.022143,0.55135,soil_suitability,-0.022603,0.541873,soil_suitability,-0.015314,0.677267,soil_suitability,0.001324,0.970188,soil_suitability,0.005825,0.867597
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,W_per_high,0.822427,0.0,mean_temp,0.059453,0.210305,mean_temp,0.053972,0.376822,mean_temp,0.006862,...,0.072694,0.302434,mean_temp,0.052368,0.455515,mean_temp,0.04102,0.545277,mean_temp,0.044171,0.511239,mean_temp,0.042305,0.526032,mean_temp,0.070813,0.26626,mean_temp,0.070729,0.263732
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,W_per_high,0.797399,0.0,mean_preci,-0.000577,0.857524,mean_preci,-0.000515,...,0.002151,0.57671,mean_preci,0.002058,0.583857,mean_preci,0.001978,0.595422,mean_preci,0.002784,0.454012,mean_preci,0.003024,0.414471,mean_preci,0.005514,0.120646,mean_preci,0.006042,0.092463
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,W_per_high,0.816201,0.0,dist_100km,-0.286112,...,-0.209913,0.095474,dist_100km,-0.19257,0.116507,dist_100km,-0.18231,0.138442,dist_100km,-0.174318,0.156694,dist_100km,-0.184401,0.131944,dist_100km,-0.186144,0.111102,dist_100km,-0.195813,0.091615
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,W_per_high,0.796371,...,-0.132002,0.018724,min_distance_uni,-0.097105,0.087154,min_distance_uni,-0.090079,0.108284,min_distance_uni,-0.080378,0.163924,min_distance_uni,-0.083341,0.147196,min_distance_uni,-0.068258,0.214713,min_distance_uni,-0.058138,0.286851
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.00005,0.964532,s_prot,0.000122,0.912495,s_prot,0.000216,0.848109,s_prot,-0.000091,0.940204,s_prot,0.000184,0.881149,s_prot,-0.000571,0.632369,s_prot,-0.000726,0.537578


In [78]:
R_high

,0
0,0.176217
1,0.178277
2,0.177097
3,0.177087
4,0.207929
5,0.210695
6,0.223657
7,0.239159
8,0.240289
9,0.242872


In [79]:
ak_high

,0
0,0.025675
1,0.026792
2,0.159729
3,0.157498
4,0.130287
5,0.094866
6,0.141186
7,0.137693
8,0.190646
9,0.169501


### Check if there is still spatial autocorrelation

#### Unskilled

In [80]:
for col in res_lag_unskilled.columns:
    res_lag_unskilled[col] = res_lag_unskilled[col].astype(float)

test_statistics = []
p_values = []

# 

for column in res_lag_unskilled.columns:
    mi_res = Moran(res_lag_unskilled[column], w)
    
    test_statistics.append(mi_res.I)
    p_values.append(mi_res.p_sim)

for i in range(len(test_statistics)):
    
    print(f"Variable {i+1}: Test Statistic = {test_statistics[i]}, P-value = {p_values[i]}")

Variable 1: Test Statistic = 0.037563377764979304, P-value = 0.003
Variable 2: Test Statistic = 0.02153969939581919, P-value = 0.017
Variable 3: Test Statistic = 0.015266094220331625, P-value = 0.047
Variable 4: Test Statistic = 0.012381280438149117, P-value = 0.063
Variable 5: Test Statistic = 0.013868477179745918, P-value = 0.043
Variable 6: Test Statistic = 0.012948057690291843, P-value = 0.054
Variable 7: Test Statistic = 0.011404631212820438, P-value = 0.056
Variable 8: Test Statistic = 0.007397484471749563, P-value = 0.13
Variable 9: Test Statistic = 0.0027792729622178143, P-value = 0.195
Variable 10: Test Statistic = 0.0031411265766562034, P-value = 0.197
Variable 11: Test Statistic = -0.02221725359915962, P-value = 0.003
Variable 12: Test Statistic = -0.022686041665317073, P-value = 0.003
Variable 13: Test Statistic = -0.021616824020367045, P-value = 0.01
Variable 14: Test Statistic = -0.021645271646247462, P-value = 0.012
Variable 15: Test Statistic = -0.021186937785909812, P-

#### Low

In [81]:
for col in res_lag_low.columns:
    res_lag_low[col] = res_lag_low[col].astype(float)

test_statistics = []
p_values = []

# 

for column in res_lag_low.columns:
    mi_res = Moran(res_lag_low[column], w)
    
    test_statistics.append(mi_res.I)
    p_values.append(mi_res.p_sim)

for i in range(len(test_statistics)):
    
    print(f"Variable {i+1}: Test Statistic = {test_statistics[i]}, P-value = {p_values[i]}")

Variable 1: Test Statistic = 0.06355224114111298, P-value = 0.001
Variable 2: Test Statistic = 0.06255020195140602, P-value = 0.001
Variable 3: Test Statistic = 0.05619040091746073, P-value = 0.001
Variable 4: Test Statistic = 0.04577436077599597, P-value = 0.001
Variable 5: Test Statistic = 0.06740964948395195, P-value = 0.001
Variable 6: Test Statistic = 0.023721313813464567, P-value = 0.008
Variable 7: Test Statistic = 0.02217395257186252, P-value = 0.014
Variable 8: Test Statistic = 0.019660345000409717, P-value = 0.024
Variable 9: Test Statistic = 0.019340935016544682, P-value = 0.013
Variable 10: Test Statistic = 0.01790821885312823, P-value = 0.031
Variable 11: Test Statistic = 0.011827989956799765, P-value = 0.056
Variable 12: Test Statistic = 0.011801287434710525, P-value = 0.077
Variable 13: Test Statistic = 0.012861587177017846, P-value = 0.05
Variable 14: Test Statistic = 0.011654830899583135, P-value = 0.055
Variable 15: Test Statistic = 0.019749924523857945, P-value = 0.0

#### Medium

In [82]:
for col in res_lag_medium.columns:
    res_lag_medium[col] = res_lag_medium[col].astype(float)

test_statistics = []
p_values = []

# 

for column in res_lag_medium.columns:
    mi_res = Moran(res_lag_medium[column], w)
    
    test_statistics.append(mi_res.I)
    p_values.append(mi_res.p_sim)

for i in range(len(test_statistics)):
    
    print(f"Variable {i+1}: Test Statistic = {test_statistics[i]}, P-value = {p_values[i]}")

Variable 1: Test Statistic = 0.146713174700422, P-value = 0.001
Variable 2: Test Statistic = 0.13777917255371583, P-value = 0.001
Variable 3: Test Statistic = 0.054142262965530395, P-value = 0.001
Variable 4: Test Statistic = 0.05241449952505028, P-value = 0.001
Variable 5: Test Statistic = 0.02778955315032897, P-value = 0.005
Variable 6: Test Statistic = 0.04759536687242907, P-value = 0.001
Variable 7: Test Statistic = 0.05014880286350576, P-value = 0.001
Variable 8: Test Statistic = 0.05500504200100928, P-value = 0.001
Variable 9: Test Statistic = 0.022501515660037375, P-value = 0.009
Variable 10: Test Statistic = 0.02457972232419516, P-value = 0.007
Variable 11: Test Statistic = 0.013547635664002867, P-value = 0.047
Variable 12: Test Statistic = 0.010778980534330442, P-value = 0.073
Variable 13: Test Statistic = 0.014413130868892033, P-value = 0.049
Variable 14: Test Statistic = 0.015985015055471525, P-value = 0.032
Variable 15: Test Statistic = 0.008749488716134651, P-value = 0.091

#### High

In [83]:
for col in res_lag_high.columns:
    res_lag_high[col] = res_lag_high[col].astype(float)

test_statistics = []
p_values = []

# 

for column in res_lag_high.columns:
    mi_res = Moran(res_lag_high[column], w)
    
    test_statistics.append(mi_res.I)
    p_values.append(mi_res.p_sim)

for i in range(len(test_statistics)):
    
    print(f"Variable {i+1}: Test Statistic = {test_statistics[i]}, P-value = {p_values[i]}")

Variable 1: Test Statistic = 0.11466023092030299, P-value = 0.001
Variable 2: Test Statistic = 0.11422515600364225, P-value = 0.001
Variable 3: Test Statistic = 0.06880354482878162, P-value = 0.001
Variable 4: Test Statistic = 0.068619441589542, P-value = 0.001
Variable 5: Test Statistic = 0.061346003912319425, P-value = 0.001
Variable 6: Test Statistic = 0.07042617544639801, P-value = 0.001
Variable 7: Test Statistic = 0.06332284359913103, P-value = 0.001
Variable 8: Test Statistic = 0.06561570718418114, P-value = 0.001
Variable 9: Test Statistic = 0.05127956607553629, P-value = 0.001
Variable 10: Test Statistic = 0.053401487421763774, P-value = 0.001
Variable 11: Test Statistic = 0.03020230618048616, P-value = 0.007
Variable 12: Test Statistic = 0.03321099851630759, P-value = 0.003
Variable 13: Test Statistic = 0.02554321314920509, P-value = 0.007
Variable 14: Test Statistic = 0.021959936916826243, P-value = 0.01
Variable 15: Test Statistic = 0.019930308584910027, P-value = 0.02
Vari

## IV with Error

### Function

ML_Error has no pseudo-Rsquared

In [84]:
def IV_w_error(df, IV, first_stage_dependent, dependent_var, controls, w):
    """
    Performs a series of IV regressions with spatial error in the second stage,
    incorporating a first-stage regression similar to twoSLS.

    Parameters:
    df (DataFrame): The DataFrame containing the data for regression.
    IV (str): The name of the instrumental variable column in the DataFrame.
    first_stage_dependent (str): The name of the dependent variable for the first stage regression.
    dependent_var (str): The name of the dependent variable for the regression.
    controls (list of str): A list of control variable names for both stages.
    w (libpysal.weights): weight matrix to control for spatial autocorrelation.

    Returns:
    Tuple of DataFrames
    First DataFrame containing the results of each second stage regression.
    Second DataFrame containing the residuals.
    Third DataFrame containing the pseudo-Rsquares.
    Fourth DataFrame containing the p-values from the AK test.
    """

    results = []
    res = []
    # Pseudo R-Squared
    pseudo_R = []
    # AKtest
    ak_pvalues = []

    for i in range(len(controls) + 1):
        subset_controls = controls[:i]

        # First Stage Regression
        df_copy = df.copy()
        controls_formula_part = " + ".join(subset_controls)
        first_stage_formula = f"{first_stage_dependent} ~ {IV}{(' + ' if controls_formula_part else '')}{controls_formula_part}"
        first_stage_model = smf.ols(first_stage_formula, data=df_copy)
        first_stage_fit = first_stage_model.fit()
        df_copy["predicted"] = first_stage_fit.predict(df_copy)

        # Second Stage Regression (Spatial Error)
        X = df_copy[["predicted"] + subset_controls]
        y = df_copy[dependent_var]
        slm = spreg.ML_Error(y.values, X.values, w,
                             name_y=dependent_var, name_x=["predicted"] + subset_controls)

        # Formatting results
        p_value = slm.output["prob"]
        coefs = slm.output["coefficients"]
        names = slm.output["var_names"]
        r = slm.pr2
        residuals = slm.u
        
        # AKtest
        ak = AKtest(slm, w)
        ak_pvalues.append(ak.p)

        results.append(names)
        results.append(coefs)
        results.append(p_value)
        res.append(list(residuals))
        pseudo_R.append(r)

    # Convert the results to DataFrame
    results_df = pd.DataFrame(results).T
    residuals_df = pd.DataFrame(res).T
    residuals_df = residuals_df.astype(float)
    pseudoR_df = pd.DataFrame(pseudo_R)
    ak_pvalues_df = pd.DataFrame(ak_pvalues)

    return results_df, residuals_df, pseudoR_df, ak_pvalues_df

### Run the Function

The error is no problem, it just tells me that the optimizing algorithm defaults to a different stopping point

In [85]:
error_unskilled_second, e_res_un, e_r2_un, e_ak_un = IV_w_error(dep_var, "log_distance_carbon_c",
                                                             "steamengines_number", "per_unskilled",
                                                             control_list, w)
error_low_second, e_res_low, e_r2_low, e_ak_low = IV_w_error(dep_var, "log_distance_carbon_c",
                                                             "steamengines_number", "per_low",
                                                             control_list, w)
error_medium_second, e_res_medium, e_r2_medium, e_ak_medium = IV_w_error(dep_var, "log_distance_carbon_c",
                                                            "steamengines_number", "per_medium",
                                                            control_list, w)
error_high_second, e_res_high, e_r2_high, e_ak_high = IV_w_error(dep_var, "log_distance_carbon_c",
                                                      "steamengines_number", "per_high",
                                                      control_list, w)

C:\Users\soere\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\optimize\_minimize.py:892: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  warn("Method 'bounded' does not support relative tolerance in x; "
C:\Users\soere\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\optimize\_minimize.py:892: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  warn("Method 'bounded' does not support relative tolerance in x; "
C:\Users\soere\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\optimize\_minimize.py:892: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  warn("Method 'bounded' does not support relative tolerance in x; "
C:\Users\soere\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\optimize\_minimize.py:892: RuntimeWarning: Method 'bounded' does not 

C:\Users\soere\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\optimize\_minimize.py:892: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  warn("Method 'bounded' does not support relative tolerance in x; "
C:\Users\soere\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\optimize\_minimize.py:892: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  warn("Method 'bounded' does not support relative tolerance in x; "
C:\Users\soere\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\optimize\_minimize.py:892: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  warn("Method 'bounded' does not support relative tolerance in x; "
C:\Users\soere\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\optimize\_minimize.py:892: RuntimeWarning: Method 'bounded' does not 

C:\Users\soere\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\optimize\_minimize.py:892: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  warn("Method 'bounded' does not support relative tolerance in x; "
C:\Users\soere\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\optimize\_minimize.py:892: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  warn("Method 'bounded' does not support relative tolerance in x; "
C:\Users\soere\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\optimize\_minimize.py:892: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  warn("Method 'bounded' does not support relative tolerance in x; "
C:\Users\soere\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\optimize\_minimize.py:892: RuntimeWarning: Method 'bounded' does not 

#### Unskilled

In [86]:
error_unskilled_second

,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,...,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob
0,CONSTANT,5.35925,0.0,CONSTANT,4.665157,0.0,CONSTANT,2.629198,0.000257,CONSTANT,2.568154,0.000263,CONSTANT,2.776536,0.000143,CONSTANT,3.447341,0.000037,CONSTANT,3.370499,...,3.919042,0.0,CONSTANT,3.910342,0.0,CONSTANT,3.829272,0.0,CONSTANT,3.816191,0.0,CONSTANT,3.668146,0.0,CONSTANT,4.521403,0.0,CONSTANT,4.572793,0.0
1,predicted,-0.593056,0.000549,predicted,-0.408905,0.001139,predicted,-0.218369,0.001007,predicted,-0.242155,0.000578,predicted,-0.252076,0.000136,predicted,-0.226137,0.000947,predicted,-0.236877,...,-0.159308,0.032757,predicted,-0.134251,0.018069,predicted,-0.132506,0.024949,predicted,-0.135769,0.020544,predicted,-0.155643,0.008608,predicted,-0.12263,0.026663,predicted,-0.122042,0.025712
2,lambda,0.731045,0.0,longitud,0.018687,0.000526,longitud,0.014029,0.007867,longitud,0.011637,0.022296,longitud,0.006701,0.226055,longitud,-0.003154,0.68191,longitud,-0.001801,...,-0.014175,0.057957,longitud,-0.007211,0.155677,longitud,-0.006935,0.168844,longitud,-0.006152,0.22965,longitud,-0.006885,0.181915,longitud,-0.011991,0.013813,longitud,-0.012057,0.013064
3,NaN,NaN,NaN,lambda,0.496071,0.022864,latitud,0.031527,0.037438,latitud,0.036401,0.017227,latitud,0.040491,0.005603,latitud,0.0385,0.008835,latitud,0.036499,...,0.034419,0.009986,latitud,0.026924,0.001318,latitud,0.027967,0.007126,latitud,0.027525,0.008035,latitud,0.03102,0.003687,latitud,0.020232,0.045809,latitud,0.019156,0.084302
4,NaN,NaN,NaN,NaN,NaN,NaN,lambda,0.426407,0.074345,soil_suitability,-0.045025,0.018154,soil_suitability,-0.045993,0.013023,soil_suitability,-0.036339,0.064129,soil_suitability,-0.038178,...,-0.022998,0.249887,soil_suitability,0.009404,0.54476,soil_suitability,0.009634,0.542227,soil_suitability,0.009379,0.55115,soil_suitability,0.005263,0.739234,soil_suitability,-0.003863,0.79566,soil_suitability,-0.00359,0.808273
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,lambda,0.371378,0.144628,mean_temp,-0.038575,0.139575,mean_temp,-0.07474,0.02753,mean_temp,-0.05521,...,-0.093817,0.010479,mean_temp,-0.068187,0.015325,mean_temp,-0.066334,0.015803,mean_temp,-0.064189,0.018832,mean_temp,-0.060812,0.025664,mean_temp,-0.081578,0.001415,mean_temp,-0.081595,0.001407
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,lambda,0.403045,0.100902,mean_preci,-0.003791,0.051659,mean_preci,-0.003805,...,-0.005392,0.007679,mean_preci,-0.005322,0.000272,mean_preci,-0.005321,0.000248,mean_preci,-0.005111,0.000471,mean_preci,-0.005125,0.000441,mean_preci,-0.006653,0.000001,mean_preci,-0.006648,0.000002
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,lambda,0.432917,0.067812,dist_100km,0.104173,...,0.059337,0.364323,dist_100km,0.029875,0.543127,dist_100km,0.03098,0.529774,dist_100km,0.035068,0.478894,dist_100km,0.040902,0.408013,dist_100km,0.038191,0.408583,dist_100km,0.037932,0.412305
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,lambda,0.401673,...,0.075434,0.009807,min_distance_uni,0.034373,0.122555,min_distance_uni,0.033592,0.127162,min_distance_uni,0.036171,0.115138,min_distance_uni,0.036869,0.107292,min_distance_uni,0.028831,0.179419,min_distance_uni,0.029693,0.163588
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.000248,0.672425,s_prot,-0.000305,0.506264,s_prot,-0.000329,0.485079,s_prot,-0.000413,0.424988,s_prot,-0.000469,0.369235,s_prot,-0.000072,0.884644,s_prot,-0.000078,0.873974


In [87]:
e_r2_un

,0
0,0.108475
1,0.156851
2,0.163057
3,0.168477
4,0.227318
5,0.251663
6,0.259304
7,0.279942
8,0.292403
9,0.295790


#### Low

In [88]:
error_low_second

,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,...,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob
0,CONSTANT,0.740015,0.166809,CONSTANT,1.737281,0.000039,CONSTANT,7.76239,0.0,CONSTANT,7.920952,0.0,CONSTANT,7.918529,0.0,CONSTANT,6.753157,0.0,CONSTANT,6.84521,...,6.177252,0.0,CONSTANT,6.200064,0.0,CONSTANT,5.72416,0.0,CONSTANT,5.924124,0.0,CONSTANT,6.090565,0.0,CONSTANT,5.088063,0.000003,CONSTANT,4.612557,0.000052
1,predicted,0.967153,0.000008,predicted,0.724528,0.000007,predicted,0.247877,0.00304,predicted,0.280324,0.001487,predicted,0.289675,0.000643,predicted,0.237861,0.005367,predicted,0.251767,...,0.208017,0.028574,predicted,0.186844,0.037441,predicted,0.193459,0.038161,predicted,0.194926,0.033408,predicted,0.200166,0.026719,predicted,0.178661,0.038887,predicted,0.172274,0.043135
2,lambda,0.882404,0.0,longitud,-0.03101,0.014324,longitud,-0.01655,0.158509,longitud,-0.012828,0.224358,longitud,-0.008098,0.511667,longitud,0.004098,0.697257,longitud,0.002358,...,0.016148,0.137557,longitud,0.012426,0.214671,longitud,0.012948,0.192185,longitud,0.011401,0.243422,longitud,0.012501,0.200741,longitud,0.016205,0.112774,longitud,0.01646,0.098505
3,NaN,NaN,NaN,lambda,0.802042,0.0,latitud,-0.097506,0.000004,latitud,-0.106241,0.0,latitud,-0.109684,0.0,latitud,-0.104531,0.0,latitud,-0.101781,...,-0.097966,0.0,latitud,-0.094184,0.0,latitud,-0.086746,0.000006,latitud,-0.088708,0.000003,latitud,-0.092533,0.000002,latitud,-0.077966,0.000062,latitud,-0.067778,0.001237
4,NaN,NaN,NaN,NaN,NaN,NaN,lambda,0.767955,0.0,soil_suitability,0.072102,0.002031,soil_suitability,0.074121,0.001423,soil_suitability,0.059504,0.014126,soil_suitability,0.061804,...,0.052396,0.037103,soil_suitability,0.031613,0.175745,soil_suitability,0.03298,0.165905,soil_suitability,0.033507,0.154403,soil_suitability,0.03583,0.125306,soil_suitability,0.046647,0.038441,soil_suitability,0.044222,0.047618
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,lambda,0.722371,0.0,mean_temp,0.011857,0.724708,mean_temp,0.070331,0.098654,mean_temp,0.044217,...,0.074977,0.113654,mean_temp,0.061787,0.18042,mean_temp,0.06757,0.130417,mean_temp,0.065497,0.135144,mean_temp,0.06572,0.131394,mean_temp,0.080463,0.056083,mean_temp,0.080602,0.054312
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,lambda,0.773315,0.0,mean_preci,0.00687,0.006454,mean_preci,0.007027,...,0.00776,0.004016,mean_preci,0.007843,0.002025,mean_preci,0.007882,0.001844,mean_preci,0.007102,0.004321,mean_preci,0.007212,0.003633,mean_preci,0.008503,0.000496,mean_preci,0.008364,0.000627
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,lambda,0.574038,0.002903,dist_100km,-0.142723,...,-0.100008,0.235093,dist_100km,-0.084446,0.293935,dist_100km,-0.089686,0.269189,dist_100km,-0.095084,0.236857,dist_100km,-0.09805,0.220312,dist_100km,-0.102,0.188117,dist_100km,-0.098016,0.203278
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,lambda,0.555254,...,-0.062322,0.101818,min_distance_uni,-0.035302,0.345195,min_distance_uni,-0.038901,0.294546,min_distance_uni,-0.048682,0.197231,min_distance_uni,-0.04988,0.184879,min_distance_uni,-0.039809,0.276981,min_distance_uni,-0.046119,0.207043
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.000634,0.397083,s_prot,-0.000611,0.394716,s_prot,-0.000659,0.367924,s_prot,-0.000365,0.63849,s_prot,-0.00023,0.771106,s_prot,-0.00071,0.355862,s_prot,-0.000642,0.398728


In [89]:
e_r2_low

,0
0,0.212796
1,0.297671
2,0.312709
3,0.330764
4,0.334857
5,0.393986
6,0.401591
7,0.411578
8,0.412146
9,0.418203


#### Medium

In [90]:
error_medium_second

,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,...,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob
0,CONSTANT,2.510331,0.0,CONSTANT,2.740927,0.0,CONSTANT,2.4584,0.0002,CONSTANT,2.445533,0.000208,CONSTANT,2.365248,0.000338,CONSTANT,2.155841,0.00446,CONSTANT,2.210397,...,1.884709,0.007593,CONSTANT,1.938072,0.003051,CONSTANT,2.704164,0.000092,CONSTANT,2.745295,0.000103,CONSTANT,2.767685,0.000112,CONSTANT,2.175172,0.00139,CONSTANT,2.191184,0.002312
1,predicted,0.2627,0.05199,predicted,0.197695,0.054215,predicted,0.164913,0.00153,predicted,0.168961,0.002273,predicted,0.179556,0.000625,predicted,0.159656,0.004102,predicted,0.168909,...,0.119256,0.046499,predicted,0.111171,0.048606,predicted,0.103894,0.070361,predicted,0.099147,0.082287,predicted,0.097845,0.082915,predicted,0.093725,0.080358,predicted,0.093971,0.075799
2,lambda,0.914833,0.0,longitud,-0.005486,0.640117,longitud,-0.006324,0.47309,longitud,-0.00648,0.459372,longitud,-0.002424,0.767472,longitud,0.000848,0.934858,longitud,-0.001102,...,0.010718,0.225851,longitud,0.006221,0.404299,longitud,0.005479,0.435231,longitud,0.006651,0.374811,longitud,0.007087,0.355895,longitud,0.006723,0.309823,longitud,0.006621,0.31285
3,NaN,NaN,NaN,lambda,0.914407,0.0,latitud,0.007533,0.573782,latitud,0.007553,0.580346,latitud,0.0048,0.713081,latitud,0.004383,0.743964,latitud,0.006353,...,0.005468,0.667212,latitud,0.008579,0.46438,latitud,-0.003628,0.769868,latitud,-0.004971,0.695359,latitud,-0.005609,0.665562,latitud,0.004507,0.712615,latitud,0.004278,0.747299
4,NaN,NaN,NaN,NaN,NaN,NaN,lambda,0.836571,0.0,soil_suitability,0.002326,0.873173,soil_suitability,0.004571,0.750031,soil_suitability,0.000661,0.965899,soil_suitability,0.002019,...,-0.007777,0.620213,soil_suitability,-0.020784,0.151692,soil_suitability,-0.022921,0.11423,soil_suitability,-0.023089,0.110736,soil_suitability,-0.022981,0.11086,soil_suitability,-0.016853,0.226364,soil_suitability,-0.016781,0.224949
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,lambda,0.833149,0.0,mean_temp,0.01765,0.396933,mean_temp,0.033844,0.228641,mean_temp,0.017208,...,0.044301,0.144182,mean_temp,0.032002,0.275602,mean_temp,0.023601,0.395619,mean_temp,0.026522,0.338075,mean_temp,0.027304,0.324151,mean_temp,0.030026,0.251485,mean_temp,0.029806,0.254237
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,lambda,0.8021,0.0,mean_preci,0.001742,0.325129,mean_preci,0.001876,...,0.002513,0.159001,mean_preci,0.002106,0.206425,mean_preci,0.001965,0.224327,mean_preci,0.002057,0.204059,mean_preci,0.002099,0.19731,mean_preci,0.002584,0.090568,mean_preci,0.002535,0.09935
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,lambda,0.861662,0.0,dist_100km,-0.087184,...,-0.052484,0.327957,dist_100km,-0.041023,0.421104,dist_100km,-0.027661,0.583127,dist_100km,-0.027923,0.581344,dist_100km,-0.027959,0.581131,dist_100km,-0.031012,0.519562,dist_100km,-0.030553,0.524865
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,lambda,0.862394,...,-0.059287,0.015563,min_distance_uni,-0.042099,0.077759,min_distance_uni,-0.036949,0.110851,min_distance_uni,-0.038601,0.106025,min_distance_uni,-0.038748,0.105622,min_distance_uni,-0.033064,0.146512,min_distance_uni,-0.032599,0.153623
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000337,0.479112,s_prot,0.000442,0.329647,s_prot,0.000505,0.263574,s_prot,0.000508,0.291003,s_prot,0.000524,0.288323,s_prot,0.000299,0.530172,s_prot,0.000304,0.519044


In [91]:
e_r2_medium

,0
0,0.021185
1,0.024936
2,0.135208
3,0.141145
4,0.193548
5,0.159346
6,0.164041
7,0.188962
8,0.253876
9,0.256189


#### High

In [92]:
error_high_second

,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,...,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob,var_names,coefficients,prob
0,CONSTANT,-1.658387,0.049603,CONSTANT,-0.966472,0.18386,CONSTANT,-0.763955,0.651751,CONSTANT,-0.72489,0.669483,CONSTANT,-1.1168,0.517353,CONSTANT,-1.492538,0.447009,CONSTANT,-1.300537,...,-2.350169,0.214312,CONSTANT,-2.281671,0.204961,CONSTANT,-1.301981,0.500271,CONSTANT,-1.427361,0.459463,CONSTANT,-1.092575,0.570924,CONSTANT,-2.7641,0.133881,CONSTANT,-1.954998,0.308315
1,predicted,0.568102,0.095237,predicted,0.41524,0.108635,predicted,0.30178,0.023282,predicted,0.317742,0.025206,predicted,0.344521,0.010548,predicted,0.309881,0.030079,predicted,0.344964,...,0.263851,0.08977,predicted,0.241929,0.106229,predicted,0.230496,0.136436,predicted,0.229577,0.133729,predicted,0.232213,0.122848,predicted,0.212046,0.136996,predicted,0.22219,0.111922
2,lambda,0.884812,0.0,longitud,-0.024101,0.359019,longitud,-0.021901,0.335908,longitud,-0.020506,0.371634,longitud,-0.00466,0.841287,longitud,-0.000087,0.997509,longitud,-0.007604,...,0.01754,0.517606,longitud,0.010254,0.653713,longitud,0.011088,0.633451,longitud,0.011586,0.604681,longitud,0.013861,0.522062,longitud,0.015168,0.43446,longitud,0.013664,0.45249
3,NaN,NaN,NaN,lambda,0.883387,0.0,latitud,0.000543,0.987366,latitud,-0.002522,0.942745,latitud,-0.01313,0.698656,latitud,-0.013984,0.686573,latitud,-0.006429,...,0.001468,0.965515,latitud,0.008377,0.795311,latitud,-0.008495,0.807258,latitud,-0.006784,0.845201,latitud,-0.013898,0.691121,latitud,0.01373,0.680286,latitud,-0.002279,0.948829
4,NaN,NaN,NaN,NaN,NaN,NaN,lambda,0.839884,0.0,soil_suitability,0.027611,0.459694,soil_suitability,0.032859,0.371196,soil_suitability,0.025173,0.525833,soil_suitability,0.030386,...,0.010742,0.791454,soil_suitability,-0.013156,0.731483,soil_suitability,-0.016105,0.678635,soil_suitability,-0.01636,0.672661,soil_suitability,-0.012615,0.742372,soil_suitability,0.004846,0.895566,soil_suitability,0.009472,0.794964
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,lambda,0.840331,0.0,mean_temp,0.079549,0.137196,mean_temp,0.111615,0.122485,mean_temp,0.050052,...,0.100491,0.204452,mean_temp,0.073901,0.346536,mean_temp,0.06743,0.37251,mean_temp,0.067075,0.368758,mean_temp,0.067217,0.364082,mean_temp,0.076896,0.272369,mean_temp,0.07231,0.296505
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,lambda,0.838604,0.0,mean_preci,0.003499,0.443862,mean_preci,0.00388,...,0.005092,0.280211,mean_preci,0.003984,0.378835,mean_preci,0.004091,0.363797,mean_preci,0.00441,0.317107,mean_preci,0.004382,0.316552,mean_preci,0.005988,0.146941,mean_preci,0.005718,0.162966
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,lambda,0.879016,0.0,dist_100km,-0.315246,...,-0.226276,0.105699,dist_100km,-0.209199,0.124284,dist_100km,-0.189152,0.167162,dist_100km,-0.185603,0.173422,dist_100km,-0.189041,0.162744,dist_100km,-0.203368,0.114058,dist_100km,-0.207948,0.102152
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,lambda,0.872534,...,-0.161534,0.011702,min_distance_uni,-0.128812,0.043794,min_distance_uni,-0.122704,0.052241,min_distance_uni,-0.116264,0.070863,min_distance_uni,-0.11745,0.066587,min_distance_uni,-0.101531,0.09566,min_distance_uni,-0.08728,0.149639
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.000464,0.708256,s_prot,-0.000187,0.876785,s_prot,-0.000184,0.880639,s_prot,-0.000335,0.794857,s_prot,0.000008,0.995103,s_prot,-0.000662,0.60148,s_prot,-0.000688,0.580982


In [93]:
e_r2_high

,0
0,0.017677
1,0.029971
2,0.072276
3,0.072421
4,0.120223
5,0.092526
6,0.112359
7,0.140572
8,0.155218
9,0.155851


### Check for spatial autocorrelation

#### Unskilled

In [94]:
e_ak_un

,0
0,4.277483e-13
1,6.411804e-02
2,1.618130e-01
3,2.432267e-01
4,1.545480e-01
5,1.151068e-01
6,1.663740e-01
7,2.667555e-01
8,7.014059e-01
9,6.809297e-01


In [95]:
test_statistics = []
p_values = []

# 

for column in e_res_un.columns:
    mi_res = Moran(e_res_un[column], w)
    
    test_statistics.append(mi_res.I)
    p_values.append(mi_res.p_sim)

for i in range(len(test_statistics)):
    
    print(f"Variable {i+1}: Test Statistic = {test_statistics[i]}, P-value = {p_values[i]}")

Variable 1: Test Statistic = 0.07706721947318299, P-value = 0.001
Variable 2: Test Statistic = 0.019967078257830135, P-value = 0.014
Variable 3: Test Statistic = 0.015057876544043178, P-value = 0.034
Variable 4: Test Statistic = 0.012567189559689997, P-value = 0.062
Variable 5: Test Statistic = 0.015545474163515753, P-value = 0.037
Variable 6: Test Statistic = 0.01720498836873812, P-value = 0.033
Variable 7: Test Statistic = 0.015110316117960336, P-value = 0.038
Variable 8: Test Statistic = 0.01212814447809978, P-value = 0.06
Variable 9: Test Statistic = 0.004186738125481887, P-value = 0.17
Variable 10: Test Statistic = 0.004488388608953298, P-value = 0.166
Variable 11: Test Statistic = -0.026269224060562785, P-value = 0.002
Variable 12: Test Statistic = -0.026841408371542173, P-value = 0.001
Variable 13: Test Statistic = -0.02646815958989193, P-value = 0.001
Variable 14: Test Statistic = -0.026848850620418706, P-value = 0.001
Variable 15: Test Statistic = -0.026613008729693226, P-valu

#### Low

In [96]:
e_ak_low

,0
0,2.597476e-65
1,3.024320e-10
2,2.851249e-08
3,1.788208e-06
4,4.717192e-13
5,2.288908e-02
6,3.986095e-02
7,6.737183e-02
8,6.235303e-02
9,7.580326e-02


In [97]:
test_statistics = []
p_values = []

# 

for column in e_res_low.columns:
    mi_res = Moran(e_res_low[column], w)
    
    test_statistics.append(mi_res.I)
    p_values.append(mi_res.p_sim)

for i in range(len(test_statistics)):
    
    print(f"Variable {i+1}: Test Statistic = {test_statistics[i]}, P-value = {p_values[i]}")

Variable 1: Test Statistic = 0.1860645422616166, P-value = 0.001
Variable 2: Test Statistic = 0.06859107621804278, P-value = 0.001
Variable 3: Test Statistic = 0.05994355556580882, P-value = 0.001
Variable 4: Test Statistic = 0.05181041635001116, P-value = 0.001
Variable 5: Test Statistic = 0.0755485114774344, P-value = 0.001
Variable 6: Test Statistic = 0.024297676740128577, P-value = 0.007
Variable 7: Test Statistic = 0.021990395280521137, P-value = 0.014
Variable 8: Test Statistic = 0.019516764226765442, P-value = 0.032
Variable 9: Test Statistic = 0.019847513769296242, P-value = 0.023
Variable 10: Test Statistic = 0.018808456937751898, P-value = 0.024
Variable 11: Test Statistic = 0.013509269387191774, P-value = 0.054
Variable 12: Test Statistic = 0.013514414093400843, P-value = 0.054
Variable 13: Test Statistic = 0.013832092440599486, P-value = 0.054
Variable 14: Test Statistic = 0.013052912031354585, P-value = 0.056
Variable 15: Test Statistic = 0.022829165067121612, P-value = 0.

#### Medium

In [98]:
e_ak_medium

,0
0,8.030457e-74
1,3.763420e-68
2,1.247504e-23
3,6.089846e-22
4,1.229513e-11
5,8.233755e-27
6,3.556844e-28
7,6.008111e-25
8,3.135966e-07
9,8.383922e-09


In [99]:
test_statistics = []
p_values = []

# 

for column in e_res_medium.columns:
    mi_res = Moran(e_res_medium[column], w)
    
    test_statistics.append(mi_res.I)
    p_values.append(mi_res.p_sim)

for i in range(len(test_statistics)):
    
    print(f"Variable {i+1}: Test Statistic = {test_statistics[i]}, P-value = {p_values[i]}")

Variable 1: Test Statistic = 0.13460387675581745, P-value = 0.001
Variable 2: Test Statistic = 0.13180049551180126, P-value = 0.001
Variable 3: Test Statistic = 0.08091145246963181, P-value = 0.001
Variable 4: Test Statistic = 0.07879187336218614, P-value = 0.001
Variable 5: Test Statistic = 0.06584107847360154, P-value = 0.001
Variable 6: Test Statistic = 0.09851815460831144, P-value = 0.001
Variable 7: Test Statistic = 0.10073734222213471, P-value = 0.001
Variable 8: Test Statistic = 0.09829499118462647, P-value = 0.001
Variable 9: Test Statistic = 0.05216584860382203, P-value = 0.001
Variable 10: Test Statistic = 0.05908401706311576, P-value = 0.001
Variable 11: Test Statistic = 0.040397388375204624, P-value = 0.001
Variable 12: Test Statistic = 0.037071231213887716, P-value = 0.002
Variable 13: Test Statistic = 0.049838193346117325, P-value = 0.001
Variable 14: Test Statistic = 0.05260858424809178, P-value = 0.001
Variable 15: Test Statistic = 0.03373287584274648, P-value = 0.002
V

#### High

In [100]:
e_ak_high

,0
0,1.540448e-27
1,1.116462e-26
2,2.639572e-15
3,2.815283e-15
4,1.202049e-15
5,1.032674e-27
6,1.424558e-27
7,6.995033e-27
8,1.324187e-20
9,1.711117e-21


In [101]:
test_statistics = []
p_values = []

# 

for column in e_res_high.columns:
    mi_res = Moran(e_res_high[column], w)
    
    test_statistics.append(mi_res.I)
    p_values.append(mi_res.p_sim)

for i in range(len(test_statistics)):
    
    print(f"Variable {i+1}: Test Statistic = {test_statistics[i]}, P-value = {p_values[i]}")

Variable 1: Test Statistic = 0.11820035053599594, P-value = 0.001
Variable 2: Test Statistic = 0.11458811678294586, P-value = 0.001
Variable 3: Test Statistic = 0.08621971541324273, P-value = 0.001
Variable 4: Test Statistic = 0.08611391692332779, P-value = 0.001
Variable 5: Test Statistic = 0.08162364868185157, P-value = 0.001
Variable 6: Test Statistic = 0.1096930490367246, P-value = 0.001
Variable 7: Test Statistic = 0.11084405395807824, P-value = 0.001
Variable 8: Test Statistic = 0.10768879954484012, P-value = 0.001
Variable 9: Test Statistic = 0.09371372953690112, P-value = 0.001
Variable 10: Test Statistic = 0.0944134371966602, P-value = 0.001
Variable 11: Test Statistic = 0.06852914907916853, P-value = 0.001
Variable 12: Test Statistic = 0.07388187401705375, P-value = 0.001
Variable 13: Test Statistic = 0.06815531172693126, P-value = 0.001
Variable 14: Test Statistic = 0.06289577627719103, P-value = 0.001
Variable 15: Test Statistic = 0.049857971603900465, P-value = 0.001
Varia

## Function GM_Lag
Makes little sense as we instrument our dependent variable --> against exclusion restriction

In [102]:
X = dep_var[control_list_2]  # Assuming control_list is a list of column names
yend = dep_var["steamengines_number"].astype(float).to_frame()
q = dep_var["log_distance_carbon_c"].to_frame()

### Unskilled

In [103]:
y_un = dep_var["per_unskilled"].to_frame()
reg_un=GM_Lag(y_un.values, X.values, yend.values, q.values,
              w=w, w_lags=1, robust="white", spat_diag = True)
reg_un.output

,var_names,coefficients,std_err,zt_stat,prob
0,CONSTANT,5.406233,2.108192,2.564394,0.010336
1,var_1,-0.000106,0.000459,-0.231405,0.817
2,var_2,-0.000284,0.000601,-0.471693,0.637146
3,var_3,0.000591,0.020376,0.02901,0.976857
4,var_4,-0.068005,0.028876,-2.355076,0.018519
5,var_5,-0.00495,0.002121,-2.33421,0.019585
6,var_6,0.054631,0.047192,1.157615,0.247021
7,var_7,0.034555,0.027989,1.234627,0.216969
8,var_8,-0.133835,0.052047,-2.571437,0.010128
9,var_9,-0.051379,0.154772,-0.331968,0.739913


In [104]:
print(reg_un.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: SPATIAL TWO STAGE LEAST SQUARES
--------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :     dep_var                Number of Observations:         267
Mean dependent var  :      3.9144                Number of Variables   :          16
S.D. dependent var  :      0.2345                Degrees of Freedom    :         251
Pseudo R-squared    :      0.5142
Spatial Pseudo R-squared:  0.5131

White Standard Errors
------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT         5.40623         2.10819         2.56439         0.01034
               var_1        -0.00011         0.00046        -0.23140         0.81700
               var_2

In [105]:
res_lag_un = reg_un.e_pred

mi_res_lag_un = Moran(res_lag_un, w)
    
print(f"P-value = {mi_res_lag_un.p_sim}")

P-value = 0.129


### Low

In [106]:
y = dep_var["per_low"].to_frame()
reg_low=GM_Lag(y.values, X.values, yend.values, q.values,
               w=w, w_lags=1, robust="white", spat_diag = True)
reg_low.output

,var_names,coefficients,std_err,zt_stat,prob
0,CONSTANT,1.490596,0.709479,2.100973,0.035643
1,var_1,-0.000547,0.000675,-0.809943,0.417973
2,var_2,0.000662,0.0009,0.736217,0.461599
3,var_3,0.025062,0.02261,1.108482,0.267654
4,var_4,0.056731,0.038368,1.478609,0.139245
5,var_5,0.005705,0.002677,2.131353,0.03306
6,var_6,-0.110558,0.068811,-1.606687,0.108123
7,var_7,-0.052768,0.039046,-1.351437,0.176556
8,var_8,0.052226,0.061578,0.84813,0.396366
9,var_9,0.039939,0.205495,0.194354,0.845899


In [107]:
print(reg_low.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: SPATIAL TWO STAGE LEAST SQUARES
--------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :     dep_var                Number of Observations:         267
Mean dependent var  :      3.0871                Number of Variables   :          16
S.D. dependent var  :      0.3199                Degrees of Freedom    :         251
Pseudo R-squared    :      0.4886
Spatial Pseudo R-squared:  0.4826

White Standard Errors
------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT         1.49060         0.70948         2.10097         0.03564
               var_1        -0.00055         0.00068        -0.80994         0.41797
               var_2

In [108]:
res_lag_low = reg_low.e_pred

mi_res_lag_low = Moran(res_lag_low, w)
    
print(f"P-value = {mi_res_lag_low.p_sim}")

P-value = 0.004


### Medium

In [109]:
y_medium = dep_var["per_medium"].to_frame()
reg_medium=GM_Lag(y_medium.values, X.values, yend.values, q.values, w=w, w_lags=1,
                  robust="white", spat_diag=True)
reg_medium.output

,var_names,coefficients,std_err,zt_stat,prob
0,CONSTANT,0.589948,1.013932,0.581841,0.560674
1,var_1,0.000165,0.000373,0.442976,0.657783
2,var_2,0.00073,0.000484,1.509049,0.131286
3,var_3,-0.013543,0.014667,-0.923305,0.355848
4,var_4,-0.002834,0.014727,-0.192456,0.847385
5,var_5,0.00077,0.000988,0.780061,0.435355
6,var_6,-0.035421,0.041519,-0.85312,0.393593
7,var_7,-0.010206,0.019464,-0.52436,0.600028
8,var_8,0.057231,0.025318,2.260513,0.023789
9,var_9,0.102453,0.122668,0.835204,0.403603


In [110]:
print(reg_medium.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: SPATIAL TWO STAGE LEAST SQUARES
--------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :     dep_var                Number of Observations:         267
Mean dependent var  :      3.1934                Number of Variables   :          16
S.D. dependent var  :      0.1781                Degrees of Freedom    :         251
Pseudo R-squared    :      0.4458
Spatial Pseudo R-squared:  0.4483

White Standard Errors
------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT         0.58995         1.01393         0.58184         0.56067
               var_1         0.00017         0.00037         0.44298         0.65778
               var_2

In [111]:
res_lag_m = reg_medium.e_pred

mi_res_lag_m = Moran(res_lag_m, w)
    
print(f"P-value = {mi_res_lag_m.p_sim}")

P-value = 0.263


### High

In [112]:
y_high = dep_var["per_high"].to_frame()
reg_high=GM_Lag(y_high.values, X.values, yend.values, q.values, w=w, w_lags=1,
                robust="white", spat_diag=True)
reg_high.output

,var_names,coefficients,std_err,zt_stat,prob
0,CONSTANT,-0.32119,0.444412,-0.72273,0.469846
1,var_1,-0.000658,0.001062,-0.619056,0.535879
2,var_2,0.000197,0.001299,0.151711,0.879415
3,var_3,-0.005283,0.042971,-0.122936,0.902158
4,var_4,0.005266,0.034573,0.152302,0.878949
5,var_5,0.001559,0.002613,0.596513,0.550833
6,var_6,-0.204263,0.104602,-1.952769,0.050847
7,var_7,-0.029864,0.056285,-0.530587,0.595705
8,var_8,0.06414,0.110236,0.581841,0.560674
9,var_9,0.010994,0.312447,0.035186,0.971932


In [113]:
print(reg_high.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: SPATIAL TWO STAGE LEAST SQUARES
--------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :     dep_var                Number of Observations:         267
Mean dependent var  :     -0.2930                Number of Variables   :          16
S.D. dependent var  :      0.4433                Degrees of Freedom    :         251
Pseudo R-squared    :      0.3621
Spatial Pseudo R-squared: omitted due to rho outside the boundary (-1, 1).

White Standard Errors
------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT        -0.32119         0.44441        -0.72273         0.46985
               var_1        -0.00066         0.00106        -0.6

In [114]:
res_lag_h = reg_high.e_pred

mi_res_lag_h = Moran(res_lag_h, w)
    
print(f"P-value = {mi_res_lag_h.p_sim}")

P-value = 0.001


C:\Users\soere\AppData\Local\Programs\Python\Python310\lib\site-packages\esda\moran.py:227: RuntimeWarning: invalid value encountered in double_scalars
  k = k_num / k_den
C:\Users\soere\AppData\Local\Programs\Python\Python310\lib\site-packages\esda\moran.py:238: RuntimeWarning: invalid value encountered in double_scalars
  return self.n / self.w.s0 * inum / self.z2ss
C:\Users\soere\AppData\Local\Programs\Python\Python310\lib\site-packages\esda\moran.py:202: RuntimeWarning: invalid value encountered in divide
  self.z /= sy


## GM_Combo
Last one

In [115]:
X = dep_var[control_list_2]
X_names = ["s_prot", "s_german", "soil_suitability",
           "mean_temp", "mean_preci", "dist_100km", "min_distance_uni",
           "prov_cap_dummy", "BER_dummy", "density"]
yend = dep_var["steamengines_number"].astype(float).to_frame()
q = dep_var["log_distance_carbon_c"].to_frame()

#### Unskilled

In [116]:
y = dep_var["per_unskilled"].to_frame()
reg=GM_Combo(y.values, X.values, yend.values, q.values, w=w, name_x=control_list_2)
reg.output

,var_names,coefficients,std_err,zt_stat,prob
0,CONSTANT,5.663017,1.388567,4.078317,0.000045
1,s_prot,-0.00013,0.000448,-0.290449,0.771473
2,s_german,-0.000304,0.00065,-0.468223,0.639625
3,soil_suitability,0.002513,0.013812,0.181942,0.855628
4,mean_temp,-0.069055,0.023731,-2.909972,0.003615
5,mean_preci,-0.005104,0.001448,-3.52351,0.000426
6,dist_100km,0.048165,0.050436,0.954982,0.339587
7,min_distance_uni,0.036594,0.024138,1.516025,0.129513
8,density,-0.143406,0.024535,-5.845034,0.0
9,s_low_stud,-0.030383,0.136749,-0.222181,0.824173


In [117]:
print(reg.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: SPATIALLY WEIGHTED 2SLS - GM-COMBO MODEL
-----------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :     dep_var                Number of Observations:         267
Mean dependent var  :      3.9144                Number of Variables   :          16
S.D. dependent var  :      0.2345                Degrees of Freedom    :         251
Pseudo R-squared    :      0.5165
Spatial Pseudo R-squared:  0.5138

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT         5.66302         1.38857         4.07832         0.00005
              s_prot        -0.00013         0.00045        -0.29045         0.77147
            s_german    

#### Low

In [118]:
y = dep_var["per_low"].to_frame()
reg=GM_Combo(y.values, X.values, yend.values, q.values, w=w, name_x=control_list_2)
reg.output

,var_names,coefficients,std_err,zt_stat,prob
0,CONSTANT,-1.050961,1.310688,-0.801839,0.422646
1,s_prot,-0.000198,0.000724,-0.274008,0.784079
2,s_german,0.000751,0.001017,0.738196,0.460396
3,soil_suitability,0.023885,0.020815,1.147495,0.251177
4,mean_temp,0.051735,0.041231,1.254765,0.209564
5,mean_preci,0.00596,0.002819,2.114064,0.03451
6,dist_100km,-0.12739,0.080752,-1.577544,0.114671
7,min_distance_uni,-0.036895,0.040434,-0.912459,0.361527
8,density,0.036327,0.03361,1.080862,0.279758
9,s_low_stud,0.057515,0.207984,0.276534,0.782138


In [119]:
print(reg.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: SPATIALLY WEIGHTED 2SLS - GM-COMBO MODEL
-----------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :     dep_var                Number of Observations:         267
Mean dependent var  :      3.0871                Number of Variables   :          16
S.D. dependent var  :      0.3199                Degrees of Freedom    :         251
Pseudo R-squared    :      0.4661
Spatial Pseudo R-squared:  0.2881

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT        -1.05096         1.31069        -0.80184         0.42265
              s_prot        -0.00020         0.00072        -0.27401         0.78408
            s_german    

#### Medium

In [120]:
y = dep_var["per_medium"].to_frame()
reg=GM_Combo(y.values, X.values, yend.values, q.values, w=w, name_x=control_list_2)
reg.output

,var_names,coefficients,std_err,zt_stat,prob
0,CONSTANT,0.665034,0.610132,1.089984,0.27572
1,s_prot,0.000238,0.000359,0.66277,0.507478
2,s_german,0.000613,0.000488,1.254926,0.209506
3,soil_suitability,-0.015742,0.010711,-1.469739,0.141632
4,mean_temp,-0.004644,0.012974,-0.357957,0.720376
5,mean_preci,0.000878,0.000718,1.222802,0.221404
6,dist_100km,-0.020827,0.0384,-0.542354,0.587575
7,min_distance_uni,-0.009297,0.01694,-0.548778,0.583158
8,density,0.071923,0.020026,3.591478,0.000329
9,s_low_stud,0.093393,0.107667,0.86742,0.385712


In [121]:
print(reg.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: SPATIALLY WEIGHTED 2SLS - GM-COMBO MODEL
-----------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :     dep_var                Number of Observations:         267
Mean dependent var  :      3.1934                Number of Variables   :          16
S.D. dependent var  :      0.1781                Degrees of Freedom    :         251
Pseudo R-squared    :      0.4454
Spatial Pseudo R-squared:  0.4542

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT         0.66503         0.61013         1.08998         0.27572
              s_prot         0.00024         0.00036         0.66277         0.50748
            s_german    

#### High

In [122]:
y = dep_var["per_medium"].to_frame()
reg=GM_Combo(y.values, X.values, yend.values, q.values, w=w, name_x=control_list_2)
reg.output

,var_names,coefficients,std_err,zt_stat,prob
0,CONSTANT,0.665034,0.610132,1.089984,0.27572
1,s_prot,0.000238,0.000359,0.66277,0.507478
2,s_german,0.000613,0.000488,1.254926,0.209506
3,soil_suitability,-0.015742,0.010711,-1.469739,0.141632
4,mean_temp,-0.004644,0.012974,-0.357957,0.720376
5,mean_preci,0.000878,0.000718,1.222802,0.221404
6,dist_100km,-0.020827,0.0384,-0.542354,0.587575
7,min_distance_uni,-0.009297,0.01694,-0.548778,0.583158
8,density,0.071923,0.020026,3.591478,0.000329
9,s_low_stud,0.093393,0.107667,0.86742,0.385712


In [123]:
print(reg.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: SPATIALLY WEIGHTED 2SLS - GM-COMBO MODEL
-----------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :     dep_var                Number of Observations:         267
Mean dependent var  :      3.1934                Number of Variables   :          16
S.D. dependent var  :      0.1781                Degrees of Freedom    :         251
Pseudo R-squared    :      0.4454
Spatial Pseudo R-squared:  0.4542

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT         0.66503         0.61013         1.08998         0.27572
              s_prot         0.00024         0.00036         0.66277         0.50748
            s_german    

## spreg.TSLS

In [124]:
from spreg import TSLS

kernel weights matrix

In [125]:
# Extracting values
coordinates = dep_var[["latitud", "longitud"]].values

# Create kernel weights matrix
kernel_weights = pysal.lib.weights.Kernel.from_array(coordinates)

# Set main diagonal to one
np.fill_diagonal(kernel_weights.full()[0], 1)

Error is of no concern and might occure due to Erblanden, as they are far away and disconnected. Note: I checked, it is not due to them

In [126]:
X = dep_var[control_list]
X_names = ["s_prot", "s_german", "longitud", "latitud", "soil_suitability",
           "mean_temp", "mean_preci", "dist_100km", "min_distance_uni",
           "prov_cap_dummy", "BER_dummy", "density"]
yend = dep_var["steamengines_number"].astype(float).to_frame()
q = dep_var["log_distance_carbon_c"].to_frame()

### Unskilled

In [127]:
y = dep_var["per_unskilled"].to_frame()
reg=TSLS(y.values, X.values, yend.values, q.values, w=w, gwk=kernel_weights,
         spat_diag=True, name_x=control_list)
reg.output

,var_names,coefficients,std_err,zt_stat,prob
0,CONSTANT,4.465945,0.75578,5.909053,0.0
1,longitud,-0.013291,0.006005,-2.213478,0.026865
2,latitud,0.022728,0.013663,1.663435,0.096225
3,soil_suitability,-0.005809,0.01614,-0.359897,0.718924
4,mean_temp,-0.0881,0.029187,-3.018482,0.00254
5,mean_preci,-0.006633,0.001651,-4.018018,0.000059
6,dist_100km,0.045872,0.053638,0.855221,0.392429
7,min_distance_uni,0.033294,0.025137,1.324498,0.185338
8,s_prot,-0.000061,0.000544,-0.112957,0.910065
9,s_german,-0.001037,0.00072,-1.441359,0.149483


In [128]:
print(reg.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: TWO STAGE LEAST SQUARES
------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :     dep_var                Number of Observations:         267
Mean dependent var  :      3.9144                Number of Variables   :          17
S.D. dependent var  :      0.2345                Degrees of Freedom    :         250
Pseudo R-squared    :      0.5165

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT         4.46594         0.75578         5.90905         0.00000
            longitud        -0.01329         0.00600        -2.21348         0.02686
             latitud         0.02273         0.01366         1.66343         0.09623
    soi

In [129]:
#Residuals
res_lag = reg.u

mi_res_lag = Moran(res_lag, w)
    
print(f"P-value = {mi_res_lag.p_sim}")
print(f"t-stat and p-value: {reg.ak_test}. if p-value >= 0.05, we have no spatial autocorrelation in our error term")

P-value = 0.142
t-stat and p-value: (0.8947438143420622, 0.3441950085588201). if p-value >= 0.05, we have no spatial autocorrelation in our error term


### Low

In [130]:
y = dep_var["per_low"].to_frame()
reg=TSLS(y.values, X.values, yend.values, q.values, w=w,
         spat_diag=True, name_x = control_list)
reg.output

,var_names,coefficients,std_err,zt_stat,prob
0,CONSTANT,4.353441,1.056319,4.121332,0.000038
1,longitud,0.016419,0.008393,1.956415,0.050416
2,latitud,-0.062849,0.019096,-3.291179,0.000998
3,soil_suitability,0.044446,0.022558,1.970337,0.0488
4,mean_temp,0.079919,0.040793,1.959132,0.050097
5,mean_preci,0.008706,0.002307,3.773247,0.000161
6,dist_100km,-0.096484,0.074967,-1.287016,0.198089
7,min_distance_uni,-0.047178,0.035133,-1.34286,0.179317
8,s_prot,-0.000651,0.00076,-0.857305,0.391276
9,s_german,0.002104,0.001006,2.091549,0.036479


In [131]:
#Residuals
res_lag = reg.u

mi_res_lag = Moran(res_lag, w)
    
print(f"P-value = {mi_res_lag.p_sim}")
print(f"t-stat and p-value: {reg.ak_test}. if p-value >= 0.05, we have no spatial autocorrelation in our error term")

P-value = 0.012
t-stat and p-value: (3.0081613871862203, 0.08284621362584724). if p-value >= 0.05, we have no spatial autocorrelation in our error term


In [132]:
print(reg.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: TWO STAGE LEAST SQUARES
------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :     dep_var                Number of Observations:         267
Mean dependent var  :      3.0871                Number of Variables   :          17
S.D. dependent var  :      0.3199                Degrees of Freedom    :         250
Pseudo R-squared    :      0.4931

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT         4.35344         1.05632         4.12133         0.00004
            longitud         0.01642         0.00839         1.95642         0.05042
             latitud        -0.06285         0.01910        -3.29118         0.00100
    soi

### Medium

In [133]:
y = dep_var["per_medium"].to_frame()
reg=TSLS(y.values, X.values, yend.values, q.values, w=w,
         spat_diag=True, name_x = control_list)
reg.output

,var_names,coefficients,std_err,zt_stat,prob
0,CONSTANT,2.168347,0.618552,3.505522,0.000456
1,longitud,0.002184,0.004914,0.444505,0.656678
2,latitud,0.007028,0.011182,0.62847,0.529696
3,soil_suitability,-0.016171,0.013209,-1.2242,0.220877
4,mean_temp,0.021013,0.023887,0.879658,0.379045
5,mean_preci,0.001815,0.001351,1.34346,0.179123
6,dist_100km,-0.020405,0.043899,-0.464825,0.642057
7,min_distance_uni,-0.029095,0.020573,-1.414261,0.157285
8,s_prot,0.000423,0.000445,0.950027,0.342099
9,s_german,0.00089,0.000589,1.510489,0.130919


In [134]:
#Residuals
res_lag = reg.u

mi_res_lag = Moran(res_lag, w)
    
print(f"P-value = {mi_res_lag.p_sim}")
print(f"t-stat and p-value: {reg.ak_test}. if p-value >= 0.05, we have no spatial autocorrelation in our error term")

P-value = 0.333
t-stat and p-value: (0.0007348272989529982, 0.9783738347829709). if p-value >= 0.05, we have no spatial autocorrelation in our error term


In [135]:
print(reg.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: TWO STAGE LEAST SQUARES
------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :     dep_var                Number of Observations:         267
Mean dependent var  :      3.1934                Number of Variables   :          17
S.D. dependent var  :      0.1781                Degrees of Freedom    :         250
Pseudo R-squared    :      0.4375

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT         2.16835         0.61855         3.50552         0.00046
            longitud         0.00218         0.00491         0.44450         0.65668
             latitud         0.00703         0.01118         0.62847         0.52970
    soi

### High

In [136]:
y = dep_var["per_high"].to_frame()
reg=TSLS(y.values, X.values, yend.values, q.values, w=w,
         spat_diag=True, name_x = control_list)
reg.output

,var_names,coefficients,std_err,zt_stat,prob
0,CONSTANT,-2.076172,1.65712,-1.25288,0.210249
1,longitud,0.002598,0.013166,0.197337,0.843564
2,latitud,0.008693,0.029958,0.290174,0.771683
3,soil_suitability,0.012258,0.035388,0.346378,0.729059
4,mean_temp,0.039117,0.063995,0.611245,0.541038
5,mean_preci,0.004164,0.00362,1.150326,0.25001
6,dist_100km,-0.217595,0.117606,-1.850205,0.064284
7,min_distance_uni,-0.071373,0.055115,-1.294997,0.195321
8,s_prot,-0.000271,0.001192,-0.227733,0.819854
9,s_german,0.001304,0.001578,0.826204,0.408688


In [137]:
#Residuals
res_lag = reg.u

mi_res_lag = Moran(res_lag, w)
    
print(f"P-value = {mi_res_lag.p_sim}")
print(f"t-stat and p-value: {reg.ak_test}. if p-value >= 0.05, we have no spatial autocorrelation in our error term")

P-value = 0.028
t-stat and p-value: (1.3309460661891923, 0.24863698136401938). if p-value >= 0.05, we have no spatial autocorrelation in our error term


In [138]:
print(reg.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: TWO STAGE LEAST SQUARES
------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :     dep_var                Number of Observations:         267
Mean dependent var  :     -0.2930                Number of Variables   :          17
S.D. dependent var  :      0.4433                Degrees of Freedom    :         250
Pseudo R-squared    :      0.3481

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT        -2.07617         1.65712        -1.25288         0.21025
            longitud         0.00260         0.01317         0.19734         0.84356
             latitud         0.00869         0.02996         0.29017         0.77168
    soi

# Wu Hausman Test

## Build some functions
First we need to run OLS, then we need to clean the data and then we need to use the betas and stds to perform the Wu Hausman Test

In [139]:
def perform_OLS(df, dependent_var, controls, independent_var):
    """
    Performs a series of Ordinary Least Squares (OLS) regressions on a given DataFrame.
    In each iteration, an additional control variable from the provided list is included.

    Parameters:
    df (DataFrame): The DataFrame containing the data for regression.
    dependent_var (str): The name of the dependent variable for the regression.
    controls (list of str): A list of control variable names. Each iteration includes an additional control variable.
    independent_var (str): The name of the main independent variable for the regression.

    Returns:
    DataFrame: A DataFrame containing the results (coefficients, significance, R-squared, standard errors) of each regression.
    """

    ols_results = []

    for i in range(len(controls) + 1):
        # Subset of control variables for this iteration
        subset_controls = controls[:i]

        # Build the formula, handling the case with no controls
        controls_formula_part = ' + '.join(subset_controls)
        if controls_formula_part:
            controls_formula_part = ' + ' + controls_formula_part

        # OLS Regression
        ols_formula = f"{dependent_var} ~ {independent_var}{controls_formula_part}"
        ols_model = smf.ols(ols_formula, data=df)
        ols_fit = ols_model.fit()

        # Function to format coefficients with significance
        def format_coef(coef, p_value):
            stars = ''
            if p_value < 0.01:
                stars = '***'
            elif p_value < 0.05:
                stars = '**'
            elif p_value < 0.1:
                stars = '*'
            return f"{coef:.5f} ({stars})"

        # Extract coefficients and significance
        ols_summary = ols_fit.summary2().tables[1]
        ols_coefs = ols_summary.apply(lambda row: format_coef(row['Coef.'], row['P>|t|']), axis=1)

        # Adding R-squared and Standard Errors
        ols_coefs.loc['R-squared'] = f"{ols_fit.rsquared:.3f}"
        ols_coefs.loc['std err'] = ols_fit.bse[independent_var]

        ols_results.append(ols_coefs)

    # Convert the results to DataFrame
    ols_df = pd.DataFrame(ols_results).T

    return ols_df

In [140]:
def clean_data(data):
    # Define the regular expression pattern to match non-numeric characters except for a dot and minus sign
    pattern = r"[^\d.-]"

    # Remove the pattern and convert to float
    cleaned_data = [float(re.sub(pattern, '', item)) for item in data]
    
    return cleaned_data

## Run the Test

In [141]:
# Run OLS first
ols_df_unskilled = perform_OLS(dep_var, "per_unskilled", control_list, "steamengines_number")
ols_df_low = perform_OLS(dep_var, "per_low", control_list, "steamengines_number")
ols_df_medium = perform_OLS(dep_var, "per_medium", control_list, "steamengines_number")
ols_df_high = perform_OLS(dep_var, "per_high", control_list, "steamengines_number")

### Unskilled

In [142]:
ols_df_unskilled

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
Intercept,4.07028 (***),3.83454 (***),3.03800 (***),3.00603 (***),3.63753 (***),4.53258 (***),4.48652 (***),4.76531 (***),4.36070 (***),4.47503 (***),4.16921 (***),3.88090 (***),3.80749 (***),3.66228 (***),4.54690 (***),4.45968 (***)
steamengines_number,-0.06437 (**),-0.06702 (**),-0.07353 (***),-0.08293 (***),-0.05510 (**),-0.04074 (),-0.04191 (),-0.03254 (),-0.01646 (),-0.01984 (),-0.05456 (**),-0.05175 (**),-0.05008 (**),-0.05018 (**),-0.05131 (**),-0.05082 (**)
R-squared,0.020,0.142,0.148,0.155,0.192,0.220,0.225,0.260,0.282,0.285,0.454,0.455,0.457,0.460,0.532,0.532
std err,0.0278,0.02606,0.026484,0.027167,0.027785,0.027779,0.027751,0.027302,0.027569,0.027712,0.024581,0.024994,0.025061,0.025039,0.023352,0.023433
longitud,NaN,0.01872 (***),0.01641 (***),0.01507 (***),0.00773 (*),-0.00563 (),-0.00524 (),-0.01171 (*),-0.01604 (**),-0.01886 (***),-0.01282 (**),-0.01247 (**),-0.01154 (*),-0.01277 (**),-0.01576 (***),-0.01576 (***)
latitud,NaN,NaN,0.01624 (),0.01874 (),0.01945 (),0.01711 (),0.01560 (),0.01275 (),0.02650 (**),0.02684 (**),0.02640 (**),0.03089 (**),0.03126 (**),0.03479 (**),0.02213 (*),0.02408 (*)
soil_suitability,NaN,NaN,NaN,-0.02486 (),-0.01823 (),-0.00571 (),-0.00593 (),0.00045 (),0.00182 (),0.00088 (),0.01908 (),0.01972 (),0.02002 (),0.01886 (),0.00564 (),0.00544 ()
mean_temp,NaN,NaN,NaN,NaN,-0.07617 (***),-0.12195 (***),-0.10952 (***),-0.12312 (***),-0.13700 (***),-0.13970 (***),-0.10426 (***),-0.10040 (***),-0.09945 (***),-0.10145 (***),-0.10950 (***),-0.11005 (***)
mean_preci,NaN,NaN,NaN,NaN,NaN,-0.00480 (***),-0.00492 (***),-0.00564 (***),-0.00753 (***),-0.00741 (***),-0.00670 (***),-0.00666 (***),-0.00631 (***),-0.00651 (***),-0.00755 (***),-0.00763 (***)
dist_100km,NaN,NaN,NaN,NaN,NaN,NaN,0.08856 (),0.03836 (),0.04371 (),0.04383 (),0.03047 (),0.02745 (),0.03113 (),0.03377 (),0.03749 (),0.03834 ()


### Low

In [143]:
ols_df_low

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
Intercept,3.04109 (***),3.47547 (***),7.44795 (***),7.51880 (***),7.19688 (***),5.50222 (***),5.56728 (***),5.29569 (***),5.51551 (***),5.30911 (***),5.51772 (***),5.45498 (***),5.65213 (***),5.82098 (***),4.79860 (***),4.36391 (***)
steamengines_number,0.01899 (),0.02387 (),0.05631 (*),0.07716 (**),0.06297 (*),0.03579 (),0.03744 (),0.02831 (),0.01957 (),0.02568 (),0.04936 (),0.04998 (),0.04550 (),0.04561 (),0.04692 (),0.04934 ()
R-squared,0.001,0.224,0.300,0.319,0.324,0.376,0.382,0.400,0.403,0.410,0.452,0.452,0.459,0.462,0.514,0.517
std err,0.038281,0.033802,0.03274,0.033268,0.03467,0.033869,0.033795,0.03353,0.034264,0.034345,0.033586,0.034177,0.034095,0.034092,0.032473,0.032488
longitud,NaN,-0.03449 (***),-0.02297 (***),-0.02000 (***),-0.01626 (***),0.00904 (),0.00848 (),0.01479 (*),0.01714 (**),0.02222 (***),0.01811 (**),0.01819 (**),0.01569 (*),0.01712 (**),0.02058 (**),0.02055 (**)
latitud,NaN,NaN,-0.08101 (***),-0.08653 (***),-0.08690 (***),-0.08247 (***),-0.08034 (***),-0.07756 (***),-0.08504 (***),-0.08564 (***),-0.08534 (***),-0.08436 (***),-0.08535 (***),-0.08946 (***),-0.07483 (***),-0.06511 (***)
soil_suitability,NaN,NaN,NaN,0.05512 (***),0.05174 (**),0.02804 (),0.02834 (),0.02212 (),0.02138 (),0.02308 (),0.01066 (),0.01080 (),0.00998 (),0.01133 (),0.02661 (),0.02564 ()
mean_temp,NaN,NaN,NaN,NaN,0.03883 (),0.12551 (***),0.10795 (***),0.12119 (***),0.12873 (***),0.13361 (***),0.10943 (***),0.11027 (***),0.10771 (***),0.11004 (***),0.11934 (***),0.11661 (***)
mean_preci,NaN,NaN,NaN,NaN,NaN,0.00909 (***),0.00926 (***),0.00996 (***),0.01099 (***),0.01076 (***),0.01028 (***),0.01029 (***),0.00935 (***),0.00958 (***),0.01078 (***),0.01037 (***)
dist_100km,NaN,NaN,NaN,NaN,NaN,NaN,-0.12509 (),-0.07619 (),-0.07910 (),-0.07932 (),-0.07020 (),-0.07086 (),-0.08074 (),-0.08382 (),-0.08812 (),-0.08389 ()


### Medium

In [144]:
ols_df_medium

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
Intercept,2.89762 (***),2.96403 (***),1.73768 (***),1.72457 (***),1.38008 (***),1.50296 (***),1.53711 (***),1.31635 (**),1.78188 (***),1.65284 (***),1.81454 (***),2.76007 (***),2.76038 (***),2.72937 (***),2.13880 (***),2.17080 (***)
steamengines_number,0.12216 (***),0.12291 (***),0.11289 (***),0.10903 (***),0.09385 (***),0.09582 (***),0.09669 (***),0.08927 (***),0.07077 (***),0.07458 (***),0.09294 (***),0.08372 (***),0.08372 (***),0.08369 (***),0.08445 (***),0.08427 (***)
R-squared,0.124,0.141,0.164,0.166,0.185,0.186,0.191,0.229,0.279,0.287,0.369,0.385,0.385,0.385,0.441,0.441
std err,0.019963,0.01981,0.019924,0.020498,0.021195,0.021545,0.021529,0.02116,0.020978,0.021022,0.020074,0.020158,0.020248,0.020284,0.01938,0.019451
longitud,NaN,-0.00527 (**),-0.00883 (***),-0.00938 (***),-0.00538 (*),-0.00721 (),-0.00750 (),-0.00237 (),0.00261 (),0.00579 (),0.00260 (),0.00142 (),0.00142 (),0.00115 (),0.00315 (),0.00315 ()
latitud,NaN,NaN,0.02501 (***),0.02603 (***),0.02564 (***),0.02532 (***),0.02644 (***),0.02870 (***),0.01287 (),0.01250 (),0.01273 (),-0.00199 (),-0.00199 (),-0.00124 (),0.00721 (),0.00650 ()
soil_suitability,NaN,NaN,NaN,-0.01020 (),-0.01382 (),-0.01210 (),-0.01195 (),-0.01700 (),-0.01857 (),-0.01750 (),-0.02713 (**),-0.02922 (**),-0.02922 (**),-0.02947 (**),-0.02064 (*),-0.02057 (*)
mean_temp,NaN,NaN,NaN,NaN,0.04155 (**),0.03527 (*),0.02605 (),0.03682 (*),0.05278 (**),0.05583 (***),0.03709 (*),0.02445 (),0.02445 (),0.02402 (),0.02939 (),0.02959 ()
mean_preci,NaN,NaN,NaN,NaN,NaN,-0.00066 (),-0.00057 (),-0.00001 (),0.00218 (*),0.00203 (),0.00166 (),0.00153 (),0.00153 (),0.00148 (),0.00217 (*),0.00220 (*)
dist_100km,NaN,NaN,NaN,NaN,NaN,NaN,-0.06566 (),-0.02591 (),-0.03207 (),-0.03221 (),-0.02514 (),-0.01522 (),-0.01523 (),-0.01467 (),-0.01715 (),-0.01746 ()


### High

In [145]:
ols_df_high

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
Intercept,-0.77133 (***),-0.56283 (***),-2.75100 (**),-2.74404 (**),-3.69570 (***),-3.02179 (**),-2.85554 (**),-3.40300 (**),-2.67556 (**),-2.80501 (**),-2.45942 (*),-1.30503 (),-1.63477 (),-1.24868 (),-2.88575 (*),-2.06898 ()
steamengines_number,0.19753 (***),0.19987 (***),0.18200 (***),0.18405 (***),0.14211 (**),0.15292 (***),0.15714 (***),0.13875 (**),0.10983 (**),0.11366 (**),0.15289 (***),0.14164 (**),0.14913 (***),0.14939 (***),0.15148 (***),0.14692 (***)
R-squared,0.052,0.079,0.091,0.091,0.115,0.119,0.139,0.177,0.196,0.197,0.258,0.262,0.273,0.279,0.348,0.354
std err,0.051669,0.051037,0.051701,0.053255,0.05498,0.055781,0.055294,0.054436,0.055123,0.055505,0.054173,0.054978,0.054816,0.054692,0.052106,0.052066
longitud,NaN,-0.01656 (***),-0.02290 (***),-0.02261 (***),-0.01155 (),-0.02161 (*),-0.02303 (*),-0.01032 (),-0.00253 (),0.00066 (),-0.00616 (),-0.00760 (),-0.00342 (),-0.00014 (),0.00539 (),0.00544 ()
latitud,NaN,NaN,0.04462 (*),0.04408 (*),0.04301 (*),0.04125 (*),0.04670 (*),0.05229 (**),0.02757 (),0.02719 (),0.02768 (),0.00971 (),0.01136 (),0.00197 (),0.02539 (),0.00714 ()
soil_suitability,NaN,NaN,NaN,0.00542 (),-0.00458 (),0.00485 (),0.00561 (),-0.00692 (),-0.00937 (),-0.00831 (),-0.02888 (),-0.03143 (),-0.03006 (),-0.02698 (),-0.00250 (),-0.00066 ()
mean_temp,NaN,NaN,NaN,NaN,0.11480 (***),0.08033 (),0.03545 (),0.06215 (),0.08710 (),0.09016 (),0.05011 (),0.03468 (),0.03896 (),0.04430 (),0.05919 (),0.06432 ()
mean_preci,NaN,NaN,NaN,NaN,NaN,-0.00362 (),-0.00320 (),-0.00179 (),0.00162 (),0.00148 (),0.00068 (),0.00051 (),0.00209 (),0.00262 (),0.00454 (),0.00531 ()
dist_100km,NaN,NaN,NaN,NaN,NaN,NaN,-0.31963 (**),-0.22106 (*),-0.23069 (*),-0.23082 (*),-0.21573 (*),-0.20361 (),-0.18709 (),-0.19412 (),-0.20100 (*),-0.20895 (*)


# SUR
Lets check for seemingly unrelated regression by calculating the correlation coefficients of our error term

## Unskilled & low skilled

In [146]:
# Iterating 
for column in res_unskilled.columns:
    # Calculate Pearson correlation coefficients and p-values
    corr, p_value = pearsonr(res_unskilled[column], res_low[column])

    # Printing the results 
    print(f"Column: {column}")
    print(f"Correlation: {corr}")
    print(f"P-value: {p_value}")
    print(f"{'Significantly different' if p_value < 0.05 else 'Not significantly different'}")
    print("-" * 66)  # Separator for readability

Column: 0
Correlation: -0.7928694515580138
P-value: 6.12772204033735e-59
Significantly different
------------------------------------------------------------------
Column: 1
Correlation: -0.7798270704876146
P-value: 7.7294082172973455e-56
Significantly different
------------------------------------------------------------------
Column: 2
Correlation: -0.8051484461637117
P-value: 4.538148273346913e-62
Significantly different
------------------------------------------------------------------
Column: 3
Correlation: -0.8053503377278705
P-value: 4.013770369488316e-62
Significantly different
------------------------------------------------------------------
Column: 4
Correlation: -0.8057142414904617
P-value: 3.2156994905657366e-62
Significantly different
------------------------------------------------------------------
Column: 5
Correlation: -0.7997341117071892
P-value: 1.1589936780357532e-60
Significantly different
------------------------------------------------------------------
Column: 

## unskilled & medium

In [147]:
# Iterating
for column in res_unskilled.columns:
    # Calculate Pearson correlation coefficients and p-values
    corr, p_value = pearsonr(res_unskilled[column], res_medium[column])

    # Printing the results
    print(f"Column: {column}")
    print(f"Correlation: {corr}")
    print(f"P-value: {p_value}")
    print(f"{'Significantly different' if p_value < 0.05 else 'Not significantly different'}")
    print("-" * 66)  # Separator for readability

Column: 0
Correlation: -0.6830003147447355
P-value: 5.012989894615125e-38
Significantly different
------------------------------------------------------------------
Column: 1
Correlation: -0.6883138483610552
P-value: 8.043570905949947e-39
Significantly different
------------------------------------------------------------------
Column: 2
Correlation: -0.7035890924998932
P-value: 3.3324701532338317e-41
Significantly different
------------------------------------------------------------------
Column: 3
Correlation: -0.7157710801763711
P-value: 3.2397125532441005e-43
Significantly different
------------------------------------------------------------------
Column: 4
Correlation: -0.6955588475626878
P-value: 6.223069783439315e-40
Significantly different
------------------------------------------------------------------
Column: 5
Correlation: -0.7107343549423972
P-value: 2.2654053752226725e-42
Significantly different
------------------------------------------------------------------
Column:

## unskilled & high

In [148]:
# Iterating 
for column in res_unskilled.columns:
    # Calculate Pearson correlation coefficients and p-values
    corr, p_value = pearsonr(res_unskilled[column], res_high[column])

    # Printing the results 
    print(f"Column: {column}")
    print(f"Correlation: {corr}")
    print(f"P-value: {p_value}")
    print(f"{'Significantly different' if p_value < 0.05 else 'Not significantly different'}")
    print("-" * 66)  # Separator for readability

Column: 0
Correlation: -0.7647956805786277
P-value: 1.617069844857497e-52
Significantly different
------------------------------------------------------------------
Column: 1
Correlation: -0.7644361429132317
P-value: 1.9279758756528185e-52
Significantly different
------------------------------------------------------------------
Column: 2
Correlation: -0.7686891727260496
P-value: 2.3595466361013595e-53
Significantly different
------------------------------------------------------------------
Column: 3
Correlation: -0.7723533750816608
P-value: 3.724140764623357e-54
Significantly different
------------------------------------------------------------------
Column: 4
Correlation: -0.7586536048053509
P-value: 3.125872609512439e-51
Significantly different
------------------------------------------------------------------
Column: 5
Correlation: -0.7847903218156291
P-value: 5.41548944972094e-57
Significantly different
------------------------------------------------------------------
Column: 6

## low and medium

In [149]:
# Iterating 
for column in res_low.columns:
    # Calculate Pearson correlation coefficients and p-values
    corr, p_value = pearsonr(res_low[column], res_medium[column])

    # Printing the results 
    print(f"Column: {column}")
    print(f"Correlation: {corr}")
    print(f"P-value: {p_value}")
    print(f"{'Significantly different' if p_value < 0.05 else 'Not significantly different'}")
    print("-" * 66)  # Separator for readability

Column: 0
Correlation: 0.23440142855583287
P-value: 0.00011055308436840122
Significantly different
------------------------------------------------------------------
Column: 1
Correlation: 0.22613602437870417
P-value: 0.00019443538964439663
Significantly different
------------------------------------------------------------------
Column: 2
Correlation: 0.2990096238129964
P-value: 6.448761273797044e-07
Significantly different
------------------------------------------------------------------
Column: 3
Correlation: 0.31879877166192433
P-value: 1.0131790462304106e-07
Significantly different
------------------------------------------------------------------
Column: 4
Correlation: 0.29712051967252806
P-value: 7.641062659952082e-07
Significantly different
------------------------------------------------------------------
Column: 5
Correlation: 0.31480372067109746
P-value: 1.48839994695423e-07
Significantly different
------------------------------------------------------------------
Column: 6

## low & high

In [150]:
# Iterating
for column in res_low.columns:
    # Calculate Pearson correlation coefficients and p-values
    corr, p_value = pearsonr(res_low[column], res_high[column])

    # Printing the results 
    print(f"Column: {column}")
    print(f"Correlation: {corr}")
    print(f"P-value: {p_value}")
    print(f"{'Significantly different' if p_value < 0.05 else 'Not significantly different'}")
    print("-" * 66)  # Separator for readability

Column: 0
Correlation: 0.5837726634452741
P-value: 8.733292809960466e-26
Significantly different
------------------------------------------------------------------
Column: 1
Correlation: 0.5828521324520258
P-value: 1.0852184279091413e-25
Significantly different
------------------------------------------------------------------
Column: 2
Correlation: 0.6442352954077679
P-value: 1.0541540886372527e-32
Significantly different
------------------------------------------------------------------
Column: 3
Correlation: 0.6552245077638958
P-value: 3.9218663484813765e-34
Significantly different
------------------------------------------------------------------
Column: 4
Correlation: 0.6471827377906643
P-value: 4.418654000880393e-33
Significantly different
------------------------------------------------------------------
Column: 5
Correlation: 0.693610964918951
P-value: 1.2475711296405095e-39
Significantly different
------------------------------------------------------------------
Column: 6
Cor

## medium & high

In [151]:
# Iterating 
for column in res_medium.columns:
    # Calculate Pearson correlation coefficients and p-values
    corr, p_value = pearsonr(res_medium[column], res_high[column])

    # Printing the results 
    print(f"Column: {column}")
    print(f"Correlation: {corr}")
    print(f"P-value: {p_value}")
    print(f"{'Significantly different' if p_value < 0.05 else 'Not significantly different'}")
    print("-" * 66)  # Separator for readability

Column: 0
Correlation: 0.6204237095787766
P-value: 8.463951293776425e-30
Significantly different
------------------------------------------------------------------
Column: 1
Correlation: 0.6183505086195452
P-value: 1.4749341327231947e-29
Significantly different
------------------------------------------------------------------
Column: 2
Correlation: 0.5888607293814216
P-value: 2.595926312249676e-26
Significantly different
------------------------------------------------------------------
Column: 3
Correlation: 0.5900249118190611
P-value: 1.9607741930996507e-26
Significantly different
------------------------------------------------------------------
Column: 4
Correlation: 0.5659761499429132
P-value: 5.167807887018957e-24
Significantly different
------------------------------------------------------------------
Column: 5
Correlation: 0.5660167451098681
P-value: 5.121358230098131e-24
Significantly different
------------------------------------------------------------------
Column: 6
Corr

## Run a SUR with IV and spatial lags
As soon as we add a second variable, even a completly random one, all variables lose signficance

In [152]:
control_list_3 = ["prov_cap_dummy"]
control_random = ["randomness"]

In [153]:
n_rows = len(dep_var)
sigma = 5
mu = 2

# Generate random numbers from standard normal distribution
random_numbers = sigma * np.random.randn(n_rows) + mu

# Add these numbers as a new column to your DataFrame
dep_var["randomness"] = random_numbers

C:\Users\soere\AppData\Local\Temp\ipykernel_4432\1162644816.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dep_var["randomness"] = random_numbers


In [154]:
# Add a constant term
dep_var["Constant"] = 1

# Define variable lists
y_var = ["per_unskilled", "per_low", "per_medium", "per_high"]
#x_var = [control_random + ['Constant'] for _ in range(4)]
x_var = [["Constant"] for _ in range(4)]
yend_var = ["steamengines_number"]  # Endogenous regressor
q_var = ["log_distance_carbon_c"]  # Instrument

# Create dictionaries for the model
bigy = {i: dep_var[y_var[i]].values.reshape((-1, 1)) for i in range(len(y_var))}

bigX = {}
for i in range(len(y_var)):
    bigX[i] = dep_var[x_var[i]].values

bigyend = {i: dep_var[yend_var].values.reshape((-1, 1)) for i in range(len(y_var))}
bigq = {i: dep_var[q_var].values.reshape((-1, 1)) for i in range(len(y_var))}

# Construct and run the SURlagIV model
results = SURlagIV(bigy, bigX, bigyend=bigyend, bigq=bigq, w=w, spat_diag=True)
print(results.summary)


REGRESSION
----------
SUMMARY OF OUTPUT: SEEMINGLY UNRELATED REGRESSIONS (SUR) - SPATIAL LAG MODEL
----------------------------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Number of Equations :           4                Number of Observations:         267
----------

SUMMARY OF EQUATION 1
---------------------
Dependent Variable  :   dep_var_1                Number of Variables   :           3
Mean dependent var  :      3.9144                Degrees of Freedom    :         264
S.D. dependent var  :      0.2341

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
          Constant_1       0.8265932       0.9548509       0.8656777       0.3866669
             end_1_1      -0.3460059       0.1376478      -2.5137054  

C:\Users\soere\AppData\Local\Temp\ipykernel_4432\1361207607.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dep_var["Constant"] = 1


In [155]:
results.b3SLS

{0: array([[ 0.8265932 ],
        [-0.34600591],
        [ 1.00414079]]),
 1: array([[-1.63142957],
        [ 0.54773917],
        [ 1.09760608]]),
 2: array([[0.27558281],
        [0.16857639],
        [0.78462177]]),
 3: array([[-0.96032955],
        [ 0.36836268],
        [ 0.76739676]])}

# Ifo

In [156]:
list_exog = ["log_distance_carbon_c",   "longitud", "latitud",
       "soil_suitability", "mean_temp", "mean_preci", "dist_100km","s_prot", "s_german", "min_distance_uni", "BER_dummy"]

X_exog = dep_var[list_exog]

y = dep_var["density"]

#y = np.log(dep_var["s_build"])

X_exog = sm.add_constant(X_exog)

model = sm.OLS(y, X_exog).fit()

# Extracting coefficients and standard errors
coefficients = model.params
standard_errors = model.bse

# Creating a DataFrame
summary_df = pd.DataFrame({'Coefficient': coefficients, 'Standard Error': standard_errors})

print(model.summary())
summary_df

                            OLS Regression Results                            
Dep. Variable:                density   R-squared:                       0.327
Model:                            OLS   Adj. R-squared:                  0.298
Method:                 Least Squares   F-statistic:                     11.26
Date:                Thu, 28 Dec 2023   Prob (F-statistic):           5.27e-17
Time:                        18:38:14   Log-Likelihood:                -148.19
No. Observations:                 267   AIC:                             320.4
Df Residuals:                     255   BIC:                             363.4
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    -1.37

,Coefficient,Standard Error
const,-1.373608,1.733156
log_distance_carbon_c,-0.035381,0.027882
longitud,0.012445,0.014320
latitud,0.015914,0.035269
soil_suitability,0.010278,0.036081
mean_temp,0.067602,0.056447
mean_preci,0.001577,0.003660
dist_100km,-0.083465,0.141003
s_prot,-0.000478,0.001117
s_german,0.001065,0.001702


In [157]:
list_exog = ["log_distance_carbon_c",   "longitud", "latitud",
       "soil_suitability", "mean_temp", "mean_preci", "dist_100km","s_prot", "s_german", "min_distance_uni", "BER_dummy"]

X_exog = dep_var[list_exog]

y = dep_var["s_build"]

#y = np.log(dep_var["s_build"])

X_exog = sm.add_constant(X_exog)

model = sm.OLS(y, X_exog).fit()

# Extracting coefficients and standard errors
coefficients = model.params
standard_errors = model.bse

# Creating a DataFrame
summary_df = pd.DataFrame({'Coefficient': coefficients, 'Standard Error': standard_errors})

print(model.summary())
summary_df

                            OLS Regression Results                            
Dep. Variable:                s_build   R-squared:                       0.371
Model:                            OLS   Adj. R-squared:                  0.343
Method:                 Least Squares   F-statistic:                     13.65
Date:                Thu, 28 Dec 2023   Prob (F-statistic):           1.76e-20
Time:                        18:38:14   Log-Likelihood:                -600.19
No. Observations:                 267   AIC:                             1224.
Df Residuals:                     255   BIC:                             1267.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    37.11

,Coefficient,Standard Error
const,37.116316,9.419972
log_distance_carbon_c,-0.025002,0.151546
longitud,-0.258647,0.077831
latitud,-0.384794,0.191691
soil_suitability,-0.015428,0.196103
mean_temp,-0.726232,0.306799
mean_preci,-0.089662,0.019893
dist_100km,-0.613248,0.766376
s_prot,0.035745,0.006072
s_german,0.006700,0.009252


In [158]:
list_exog = ["log_distance_carbon_c",   "longitud", "latitud",
       "soil_suitability", "mean_temp", "mean_preci", "dist_100km","s_prot", "s_german", "min_distance_uni", "BER_dummy"]

X_exog = dep_var[list_exog]

y = dep_var["s_low_stud"]

#y = np.log(dep_var["s_build"])

X_exog = sm.add_constant(X_exog)

model = sm.OLS(y, X_exog).fit()

# Extracting coefficients and standard errors
coefficients = model.params
standard_errors = model.bse

# Creating a DataFrame
summary_df = pd.DataFrame({'Coefficient': coefficients, 'Standard Error': standard_errors})

print(model.summary())
summary_df

                            OLS Regression Results                            
Dep. Variable:             s_low_stud   R-squared:                       0.425
Model:                            OLS   Adj. R-squared:                  0.400
Method:                 Least Squares   F-statistic:                     17.10
Date:                Thu, 28 Dec 2023   Prob (F-statistic):           3.50e-25
Time:                        18:38:14   Log-Likelihood:                 306.03
No. Observations:                 267   AIC:                            -588.1
Df Residuals:                     255   BIC:                            -545.0
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     1.36

,Coefficient,Standard Error
const,1.369471,0.316236
log_distance_carbon_c,-0.006612,0.005088
longitud,0.003604,0.002613
latitud,-0.024736,0.006435
soil_suitability,0.006701,0.006583
mean_temp,-0.006344,0.010299
mean_preci,0.000096,0.000668
dist_100km,-0.015782,0.025728
s_prot,0.000976,0.000204
s_german,0.001988,0.000311


In [159]:
list_exog = ["log_distance_carbon_c",   "longitud", "latitud",
       "soil_suitability", "mean_temp", "mean_preci", "dist_100km","s_prot", "s_german", "min_distance_uni", "BER_dummy"]

X_exog = dep_var[list_exog]

y = dep_var["s_high_stud"]

#y = np.log(dep_var["s_build"])

X_exog = sm.add_constant(X_exog)

model = sm.OLS(y, X_exog).fit()

# Extracting coefficients and standard errors
coefficients = model.params
standard_errors = model.bse

# Creating a DataFrame
summary_df = pd.DataFrame({'Coefficient': coefficients, 'Standard Error': standard_errors})

print(model.summary())
summary_df

                            OLS Regression Results                            
Dep. Variable:            s_high_stud   R-squared:                       0.169
Model:                            OLS   Adj. R-squared:                  0.133
Method:                 Least Squares   F-statistic:                     4.721
Date:                Thu, 28 Dec 2023   Prob (F-statistic):           1.38e-06
Time:                        18:38:14   Log-Likelihood:                 658.41
No. Observations:                 267   AIC:                            -1293.
Df Residuals:                     255   BIC:                            -1250.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     0.06

,Coefficient,Standard Error
const,0.062946,0.084497
log_distance_carbon_c,-0.001720,0.001359
longitud,-0.000801,0.000698
latitud,-0.000202,0.001719
soil_suitability,-0.002874,0.001759
mean_temp,-0.000066,0.002752
mean_preci,-0.000356,0.000178
dist_100km,-0.002015,0.006874
s_prot,0.000106,0.000054
s_german,0.000008,0.000083


In [160]:
list_exog = ["log_distance_carbon_c",   "longitud", "latitud",
       "soil_suitability", "mean_temp", "mean_preci", "dist_100km","s_prot", "s_german", "min_distance_uni", "BER_dummy"]

X_exog = dep_var[list_exog]

y = dep_var["s_agri"]

#y = np.log(dep_var["s_build"])

X_exog = sm.add_constant(X_exog)

model = sm.OLS(y, X_exog).fit()

# Extracting coefficients and standard errors
coefficients = model.params
standard_errors = model.bse

# Creating a DataFrame
summary_df = pd.DataFrame({'Coefficient': coefficients, 'Standard Error': standard_errors})

print(model.summary())
summary_df

                            OLS Regression Results                            
Dep. Variable:                 s_agri   R-squared:                       0.415
Model:                            OLS   Adj. R-squared:                  0.389
Method:                 Least Squares   F-statistic:                     16.42
Date:                Thu, 28 Dec 2023   Prob (F-statistic):           2.78e-24
Time:                        18:38:14   Log-Likelihood:                 726.49
No. Observations:                 267   AIC:                            -1429.
Df Residuals:                     255   BIC:                            -1386.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    -0.36

,Coefficient,Standard Error
const,-0.366011,0.065480
log_distance_carbon_c,0.001589,0.001053
longitud,-0.000065,0.000541
latitud,0.007967,0.001332
soil_suitability,-0.001817,0.001363
mean_temp,-0.001806,0.002133
mean_preci,-0.000255,0.000138
dist_100km,0.006572,0.005327
s_prot,-0.000012,0.000042
s_german,0.000014,0.000064


In [161]:
list_exog = ["log_distance_carbon_c",   "longitud", "latitud",
       "soil_suitability", "mean_temp", "mean_preci", "dist_100km","s_prot", "s_german", "min_distance_uni", "BER_dummy"]

X_exog = dep_var[list_exog]

y = dep_var["s_mill"]

#y = np.log(dep_var["s_build"])

X_exog = sm.add_constant(X_exog)

model = sm.OLS(y, X_exog).fit()

# Extracting coefficients and standard errors
coefficients = model.params
standard_errors = model.bse

# Creating a DataFrame
summary_df = pd.DataFrame({'Coefficient': coefficients, 'Standard Error': standard_errors})

print(model.summary())
summary_df

                            OLS Regression Results                            
Dep. Variable:                 s_mill   R-squared:                       0.449
Model:                            OLS   Adj. R-squared:                  0.425
Method:                 Least Squares   F-statistic:                     18.88
Date:                Thu, 28 Dec 2023   Prob (F-statistic):           1.82e-27
Time:                        18:38:14   Log-Likelihood:                -269.29
No. Observations:                 267   AIC:                             562.6
Df Residuals:                     255   BIC:                             605.6
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    30.45

,Coefficient,Standard Error
const,30.450690,2.727852
log_distance_carbon_c,0.069743,0.043885
longitud,-0.044363,0.022538
latitud,-0.486727,0.055510
soil_suitability,-0.038318,0.056788
mean_temp,-0.450645,0.088843
mean_preci,-0.005820,0.005761
dist_100km,0.314488,0.221929
s_prot,0.001898,0.001758
s_german,0.004231,0.002679
